PROCESS SUMMARY: I will read the data into a dataframe and

1. Drop the '**ID**' column as I will be looking at each transaction on its own, without considering what happened earlier.
2. Replace categorical data with seperate columns for each category.
3. Map unipolar binary values to {-1, 1} -- this set is more suitable for the tanh(.) activation function that I'll be using.
4. Normalize the remaining features to** zero-mean** and **unit-variance** to help speed up training.
5. **Clip** the normalized features to the range[-1, 1] to deemphasize outlying values.
6. Split data into 4 dataframes (features_train, labels_train, features_test, labels_test), with 30% of the data going to the test dataframes.
   Training dataframes contain 21,000 rows (16,355 normal, 4,645 default).
   Testing dataframes contain 9,000 rows (7,009 normal, 1,991 default)
    
7. Prepare balanced undersampled training dataframes by randomly choosing:
   4,645 normal cases from features_train/labels_train and combining them with the 4,645 default cases
    
8. Prepare balanced oversampled training dataframes by combining:
   16,355 normal cases with N repetetions of the 4,645 default cases, where N=ROUND(16,355/4,645)
   
9. Prepare balanced SMOTE-oversampled training dataframes by combining:
   16,355 normal cases with 16,355 SMOTE-created default cases
    
10. The test dataframes for both undersampled and oversampled cases are exactly the same

11. Replace the single output with two, one for each class. The higher of the two outputs will be considered the wining class while testing. This methodology avoids the work required to find the single output threshold at which the classification switches from one class to the other.

12. The classification metrics of iterest for this highly imbalanced dataset are:
    precision =  tp / (tp + fp),
    recall =  tp / (tp + fn),
    f1 =  2(precision)(recall) / (precision + recall)
      
    It is conjectured that fewer False Positives are preferable over a few more False Negatives

In [ ]:
import numpy as np, pandas as pd
from   sklearn.preprocessing   import MinMaxScaler, StandardScaler
from   sklearn.model_selection import train_test_split
import random, warnings
warnings.filterwarnings('ignore')

random_seed = 29     #seed the pseudo-random generator so that the results are reproducible
random.seed(random_seed)
np.random.seed(random_seed)

In [ ]:
data = pd.read_csv("../input/UCI_Credit_Card.csv")

Drop the '**ID**' column as I will be looking at each transaction on its own, without considering what happened earlier.
Replace categorical data with seperate columns for each category.
Map unipolar binary values to {-1, 1} -- this set is more suitable for the tanh(.) activation function that I'll be using.
Normalize the remaining features to** zero-mean** and **unit-variance** to help speed up training.
**Clip** the normalized features to the range[-1, 1] to deemphasize outlying values.

In [ ]:
data = data.drop('ID', axis=1)  # drop first column
data = pd.get_dummies(data, columns=['EDUCATION','MARRIAGE']) # replace categorical data with seperate columns for each category

# map unipolar-binry to {-1,1}
for feature in ['EDUCATION_0', 'EDUCATION_1', 'EDUCATION_2', 'EDUCATION_3', 'EDUCATION_4', 'EDUCATION_5', 'EDUCATION_6',
                'MARRIAGE_0', 'MARRIAGE_1', 'MARRIAGE_2', 'MARRIAGE_3', 'default.payment.next.month']:
    data[feature] = MinMaxScaler(feature_range = (-1, 1)).fit_transform(data[feature].reshape(-1, 1))
    
# map number to [-1,1]
for feature in ['SEX', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']:
    data[feature] = MinMaxScaler(feature_range = (-1, 1)).fit_transform(data[feature].reshape(-1, 1))
    
# normalize to zero mean, unit variance 
for feature in ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'BILL_AMT1',
                'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']:
    data[feature] = StandardScaler().fit_transform(data[feature].reshape(-1, 1))

# clip to unit variance, i.e. [-1,1]    
data = data.clip(-1.0, 1.0)

In [ ]:
def split_into_train_test_sets(df, test_data_fraction):
    df_features = df.ix[:, df.columns != "default.payment.next.month"]
    df_labels   = df.ix[:, df.columns == "default.payment.next.month"]
    df_features_train, df_features_test, df_labels_train, df_labels_test = train_test_split(df_features, df_labels, stratify=df_labels, test_size=test_data_fraction, random_state=random_seed)
        
    print('n_train', len(df_labels_train), '\nn_test ', len(df_labels_test))
    return df_features_train, df_features_test, df_labels_train, df_labels_test

# create training and testing dataframes
df_features_train, df_features_test, df_labels_train, df_labels_test = split_into_train_test_sets(data, test_data_fraction=0.3)

n_normal_test = len(df_labels_test[df_labels_test["default.payment.next.month"] == -1])
n_default_test  = len(df_labels_test[df_labels_test["default.payment.next.month"] ==  1])

normal_indices_train = np.array(df_labels_train[df_labels_train["default.payment.next.month"] == -1].index)
default_indices_train  = np.array(df_labels_train[df_labels_train["default.payment.next.month"] ==  1].index)
    
n_normal_train       = len(normal_indices_train)
n_default_train        = len(default_indices_train)

print('n_normal__train', n_normal_train, n_normal_train / (n_normal_train + n_default_train), '\nn_default_train', n_default_train, n_default_train / (n_normal_train + n_default_train))
print('n_normal__test', n_normal_test,  n_normal_test  / (n_normal_test  + n_default_test ), '\nn_default_test', n_default_test,  n_default_test  / (n_normal_test  + n_default_test ))

Make balanced training dataset through **UNDERsampling**.
Take a portion of majority class and all of minority class

In [ ]:
# make balanced training dataset through UNDERsampling. take a portion of majority class and all of minority class, but only in train set
def undersample_train(df_features_train, df_labels_train, normal_indices_train, default_indices_train, n_default_train): 
    
    normal_indices_train_undersample = np.array(np.random.choice(normal_indices_train, n_default_train, replace=False))
    
    indices_train_undersample        = np.concatenate([normal_indices_train_undersample, default_indices_train])

    df_features_train_undersample    = df_features_train.loc[indices_train_undersample, :]
    df_labels_train_undersample      = df_labels_train.loc[  indices_train_undersample, :]

    n_normal_train_undersample       = len(df_labels_train_undersample[df_labels_train_undersample["default.payment.next.month"] == -1])

    print('n_normal__train_undersample', n_normal_train_undersample, n_normal_train_undersample / (n_normal_train_undersample + n_default_train),
        '\nn_default_train            ', n_default_train,              n_default_train              / (n_normal_train_undersample + n_default_train))
    
    return df_features_train_undersample, df_labels_train_undersample

df_features_train_undersample, df_labels_train_undersample = undersample_train(df_features_train, df_labels_train, normal_indices_train, default_indices_train, n_default_train)

Make balanced training dataset through **OVERsampling**.
Take all of majority class and repetetions of minority class

In [ ]:
# make balanced training dataset through OVERsampling. take all of majority class and repetetions of minority class, but only in train set
def oversample_train(df_features_train, df_labels_train, normal_indices_train, default_indices_train, n_normal_train, n_default_train): 
    
    # oversample default transactions only
    n_default_repetetions_fraction, n_default_repetetions_int = np.modf(n_normal_train / n_default_train)
    
    default_indices_train_oversample_int = default_indices_train
    for i in range(int(n_default_repetetions_int) - 1):
        default_indices_train_oversample_int = np.concatenate([default_indices_train_oversample_int, default_indices_train])
    
    default_indices_train_undersample_fraction = np.array(np.random.choice(default_indices_train, int(n_default_repetetions_fraction * n_default_train), replace=False))
    
    indices_train_oversample     = np.concatenate([normal_indices_train, default_indices_train_oversample_int, default_indices_train_undersample_fraction])
    
    df_features_train_oversample = df_features_train.loc[indices_train_oversample, :]
    df_labels_train_oversample   = df_labels_train.loc[  indices_train_oversample, :]

    n_default_train_oversample     = len(df_labels_train_oversample[df_labels_train_oversample["default.payment.next.month"] == 1])

    print('n_normal__train           ', n_normal_train,           n_normal_train           / (n_normal_train + n_default_train_oversample),
        '\nn_default_train_oversample', n_default_train_oversample, n_default_train_oversample / (n_normal_train + n_default_train_oversample))
    
    return df_features_train_oversample, df_labels_train_oversample

df_features_train_oversample, df_labels_train_oversample = oversample_train(df_features_train, df_labels_train, normal_indices_train, default_indices_train, n_normal_train, n_default_train)

Make balanced training dataset through **SMOTE OVERsampling**.
Take all of majority class and SMOTE version of the minority class

In [ ]:
# make balanced training dataset through SMOTE OVERsampling. take all of majority class and SMOTE of minority class, but only in train set
from imblearn.over_sampling import SMOTE
def smote_train(df_features_train, df_labels_train, n_normal_train): 
    
    # oversample default transactions only
    
    features_train_smote, labels_train_smote = SMOTE(ratio='minority', random_state=random_seed).fit_sample(df_features_train, df_labels_train)

    df_features_train_smote = pd.DataFrame(features_train_smote,columns=list(df_features_train.columns))
    df_labels_train_smote   = pd.DataFrame(labels_train_smote,  columns=list(df_labels_train.columns  ))
    
    n_default_train_smote = len(df_labels_train_smote[df_labels_train_smote["default.payment.next.month"] == 1])

    print('n_normal__train      ', n_normal_train,           n_normal_train / (n_normal_train + n_default_train_smote),
        '\nn_default_train_smote', n_default_train_smote, n_default_train_smote / (n_normal_train + n_default_train_smote))
    
    return df_features_train_smote, df_labels_train_smote

df_features_train_smote, df_labels_train_smote = smote_train(df_features_train, df_labels_train, n_normal_train)

**Select the training set** by un-commenting only one of the following three lines of code.
Note that the test set remains the same for all thre training sets.

In [ ]:
#features_train, features_test, labels_train, labels_test = df_features_train_undersample.values, df_features_test.values, df_labels_train_undersample.values, df_labels_test.values
features_train, features_test, labels_train, labels_test = df_features_train_oversample.values, df_features_test.values, df_labels_train_oversample.values, df_labels_test.values
#features_train, features_test, labels_train, labels_test = df_features_train_smote.values, df_features_test.values, df_labels_train_smote.values, df_labels_test.values

**Replace the single output with two, one for each class.** The higher of the two outputs will be considered the wining class while testing. This methodology avoids the work required to find the single output threshold at which the classification switches from one class to the other.

In [ ]:
labels_train_opposite = np.multiply(labels_train, -1)
labels_train          = np.concatenate((labels_train, labels_train_opposite), axis=1)

labels_test_opposite  = np.multiply(labels_test,  -1)
labels_test           = np.concatenate((labels_test,  labels_test_opposite ), axis=1)

The following function will be used for predicting on test data and **calculating metrics**

In [ ]:
def classification_metrics(features_test, labels_test):
    preds_test         = pred.eval({x: features_test})
    correct_preds_test = tf.to_float(tf.equal(tf.argmax(preds_test, 1), tf.argmax(labels_test, 1)))

    # convert 2-output {-1, 1} format to single ouput {0, 1} format to aid in metrics' calculations
    labels_test_one_col_0_1 = tf.to_float((labels_test[:,0] + 1) / 2)
    # recreate prediction in single ouput {0, 1} format to aid in metrics' calculations
    preds_test_one_col_0_1  = tf.to_float(tf.equal(labels_test_one_col_0_1, correct_preds_test))
    
    tn = tf.count_nonzero((labels_test_one_col_0_1 - 1) * (preds_test_one_col_0_1 - 1))
    fp = tf.count_nonzero((labels_test_one_col_0_1 - 1) *  preds_test_one_col_0_1     )
    fn = tf.count_nonzero( labels_test_one_col_0_1      * (preds_test_one_col_0_1 - 1))
    tp = tf.count_nonzero( labels_test_one_col_0_1      *  preds_test_one_col_0_1     )

    accuracy  =              (tp + tn) / (tp + tn + fp + fn)
    precision =                     tp / (tp + fp)
    recall    =                     tp / (tp + fn)
    f1        = 2 * precision * recall / (precision + recall)
        
    return 'tp {:1d} tn {:1d} fp {:1d} fn {:1d} ac {:8.6f} pr {:8.6f} rc {:8.6f} f1 {:8.6f}'.format(
        sess.run(tp), sess.run(tn), sess.run(fp), sess.run(fn), sess.run(accuracy),
        sess.run(precision), sess.run(recall), sess.run(f1))

Now use the an already trained lightweight multilayer neural net (LWN) for making predictions and displaying classification metrics

In [ ]:
import tensorflow as tf

# Network layer sizes
n = {'in': 32, 'h1': 512, 'h2': 256, 'h3': 128, 'h4': 64, 'h5': 32, 'h6': 16, 'h7': 8, 'h8': 4, 'ot': 2}
weights, biases, layer = {}, {}, {}

# tf Graph input
x = tf.placeholder(tf.float32, shape=(None, n['in']))
y = tf.placeholder(tf.float32, shape=(None, n['ot']))

# the following weights and biases are from a network trained on OVERSAMPLED training data
weights['h1']=[[0,0,1,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,-1,-1,0,1,0,0,1,-1,0,-1,0,-1,0,0,0,1,0,0,1,1,0,0,0,1,0,-1,0,1,0,1,0,-1,0,1,0,1,0,-1,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,-1,-1,0,0,-1,0,0,1,1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,-1,0,-1,0,1,0,0,0,0,1,-1,0,-1,0,1,-1,0,0,0,0,0,0,0,1,1,-1,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,-1,1,-1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,-1,0,0,0,-1,1,0,0,-1,0,-1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,-1,0,-1,1,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,-1,0,0,-1,-1,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,1,-1,0,0,0,1,0,1,-1,0,0,1,1,0,0,0,0,0,0,0,0,1,-1,0,1,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,1,1,0,0,1,0,0,0,1,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,-1,1,1,1,0,-1,1,0,0,0,0,0,0,0,-1,0,0,0,-1,1,-1,0,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,1,1,-1,0,0,-1,1,0,0,-1,0,0,1,-1,1,0,0,0,-1,-1,0,0,0,1,-1,1,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,-1,-1,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,1,0,1,1,0,0,-1,1,0,-1,-1,1,0,0,1,0,-1,0,1,0],[1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,1,1,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,-1,0,1,1,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,-1,1,0,1,1,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,-1,0,0,-1,-1,0,0,0,0,-1,0,0,-1,1,0,0,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],[0,-1,0,-1,0,-1,0,1,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,-1,-1,0,0,0,-1,0,1,0,1,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,-1,1,1,0,-1,0,0,0,-1,-1,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,-1,-1,0,-1,-1,0,-1,0,0,0,1,0,0,0,0,0,-1,-1,0,-1,-1,1,0,-1,0,1,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,-1,0,1,1,-1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,1,0,0,-1,0,0,1,1,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,-1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,1,-1,0,0,1,-1,0,0,0,0,-1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,1,0,-1,0,0,1,0,0,-1,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,-1,-1,0,1,0,0,1,1,-1,0,0,1,1,0,-1,0,1,0,-1,0,0,-1,0,0,1,1,1,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,-1,1,0,-1,0,0,0,0,1,0,0,-1,1,0,0,0,-1,0,0,-1,-1,-1,-1,-1,-1,0,1,0,0,1,0,0,0,0,0,0,-1,-1,1,0,-1,-1,0,-1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,1,1,0,-1,0,-1,0,0,0,1,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,1,0,1,0,1,-1,0,0,1,0],[1,1,0,1,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,-1,1,-1,0,1,0,1,0,-1,-1,-1,0,-1,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,1,0,1,0,0,0,0,0,-1,1,0,0,1,1,0,0,0,0,-1,1,-1,0,0,0,0,-1,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,1,1,-1,0,0,-1,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,-1,-1,1,0,0,0,0,0,0,1,0,0,-1,1,0,-1,0,0,0,0,0,1,1,0,0,0,0,-1,0,1,0,-1,0,0,1,1,1,1,0,1,1,0,0,-1,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,1,1,0,-1,0,1,-1,-1,0,1,-1,0,0,0,0,-1,1,0,0,1,0,0,0,-1,-1,0,0,-1,0,0,0,0,1,0,1,-1,1,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,-1,-1,1,0,1,-1,0,-1,-1,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,1,1,0,0,0,0,-1,1,-1,1,-1,1,0,1,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,-1,1,0,0,-1,1,1,1,0,0,1,1,0,0,-1,0,0,-1,1,1,1,1,0,0,1,-1,1,0,0,0,0,0,-1,1,0,1,1,-1,1,0,0,0,1,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,-1,0,0,0,1,0,0,1,0,1,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,-1,1,0,0,-1,0,0,1,1,0,0,-1,0,1,1,-1,0,0,0,0,0,1],[0,0,0,0,-1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,1,1,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,1,-1,0,0,0,-1,-1,0,0,0,0,1,0,0,1,1,-1,0,0,0,1,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,-1,0,0,1,1,1,0,-1,0,0,-1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,1,1,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,1,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,1,0,-1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,1,-1,1,0,0,1,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,1,1,0,0,0,0,1,0,1,0,-1,0,0,0,0,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,-1,-1,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,1,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,-1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,0,-1,-1,0,1,0,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0],[0,1,0,-1,0,0,0,0,0,-1,0,1,0,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,-1,1,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,1,0,1,0,0,0,0,1,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,-1,0,0,1,0,-1,0,0,0,1,1,1,0,1,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,1,1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0],[0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,1,0,0,0,0,0,1,0,0,1,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,1,1,0,-1,0,0,-1,0,1,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,-1,0,1,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,1,0,0],[0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,-1,0,1,0,0,0,0,-1,0,1,0,0,-1,-1,1,-1,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,-1,0,-1,-1,0,0,0,-1,-1,0,0,0,-1,-1,0,0,0,0,1,0,0,0,-1,-1,-1,1,0,-1,1,0,0,0,0,-1,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,-1,0,0,0,1,0,1,0,0,0,1,0,-1,-1,0,0,0,-1,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,-1,-1,-1,-1,-1,0,0,0,1,0,0,-1,0,0,0,-1,0,0,-1,0,0,1,0,1,0,1,-1,1,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,1,1,-1,-1,0,1,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,1,0,-1,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,-1,-1,-1,1,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,-1,1,0,0,0,1,1,0,0,-1,0,0,0,0,-1,-1,0,-1,1,0,0,-1,0,0,0,-1,0,-1,1,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,0,1,1,-1,0,0,-1,-1,0,0,0,0,0,0,0],[0,0,1,0,0,0,-1,0,1,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,1,-1,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,1,-1,0,0,1,0,0,-1,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,1,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,1,0,1,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,1,-1,1,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,-1,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,-1,-1,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,1,1,1,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,1,0,0,1,1,0,1,0,-1,0],[0,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,-1,1,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,-1,0,1,0,0,0,0,0,0,0,1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,1,1,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,1,0,1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,1,0,-1,1,-1,-1,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,-1,0,-1,-1,0,-1,-1,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,-1,1,-1,1,0,1,0,0,-1,0,0,1,-1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,-1,-1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,1,0,1,-1,0,0,1,0,0,0,0,0,-1,0,0,-1,1,-1,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,-1,1,0,0,1,0,0,-1,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,1,-1,1,-1,0,0,0,0,0,0,0,0,1,0,-1,1,0,-1,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0],[0,0,-1,0,0,1,1,0,0,0,-1,0,0,0,0,1,0,0,-1,-1,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,1,1,0,0,-1,0,-1,0,0,1,0,-1,0,0,0,1,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,0,-1,0,-1,0,0,0,0,0,1,-1,0,0,1,0,-1,0,1,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,-1,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,1,-1,0,0,0,-1,0,1,0,0,0,0,1,-1,1,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,-1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,0,-1,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0],[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,1,0,1,1,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,-1,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,1,0,0,0,-1,1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,-1,-1,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,1,-1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,1,0,0,0,-1,1,0,0,0,-1,0,-1,0,0,1,0,0,1,0,0,0,-1,-1,1,0,1,0,-1,0,0,-1,1,0,-1,0,0,0,0,0,0,0,-1,0,1,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,-1,1,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,-1,0,1,0,0,0,0,-1,1,1,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,1,0,1,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,-1,0,0,0,0],[-1,0,1,0,0,0,-1,-1,1,0,0,-1,0,-1,1,-1,0,1,0,0,0,0,0,1,-1,0,-1,0,0,1,-1,-1,0,1,1,1,1,-1,1,-1,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,-1,-1,0,1,0,0,0,0,-1,-1,-1,0,0,1,-1,0,0,-1,0,0,1,-1,0,-1,0,1,1,1,1,1,0,0,0,1,0,0,0,0,-1,1,-1,0,1,0,0,0,-1,1,0,0,1,0,0,1,0,-1,0,0,0,0,0,1,-1,0,1,0,0,1,1,1,-1,-1,0,1,0,-1,0,0,0,1,-1,-1,-1,0,0,0,0,-1,1,0,1,0,0,-1,0,0,1,-1,0,0,0,1,0,-1,0,0,-1,0,0,-1,0,-1,0,0,0,-1,0,1,0,0,0,0,0,1,1,-1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,-1,-1,-1,0,1,-1,0,0,0,1,-1,0,-1,0,0,0,1,0,1,0,-1,0,0,-1,0,0,1,-1,0,0,0,-1,-1,0,-1,-1,1,0,1,0,1,-1,1,-1,1,0,1,0,1,-1,1,0,1,-1,-1,1,1,1,0,-1,-1,0,0,0,0,0,1,0,-1,1,-1,1,0,1,0,0,0,-1,0,1,-1,0,0,1,1,-1,1,0,1,0,1,0,0,-1,0,0,0,0,0,-1,1,0,0,1,0,0,1,-1,1,-1,0,0,0,0,0,0,-1,-1,1,0,0,-1,0,-1,1,-1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,-1,0,-1,-1,0,1,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,-1,1,0,-1,-1,0,0,0,-1,-1,-1,0,-1,0,0,1,0,-1,0,0,0,0,1,0,-1,0,0,1,1,0,0,-1,0,0,-1,-1,1,1,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,-1,-1,0,-1,1,0,-1,0,0,0,0,1,1,0,0,0,0,0,-1,0,1,0,1,0,1,0,0,0,-1,0,0,0,0,1,1,0,-1,0,1,-1,0,1],[-1,-1,-1,1,1,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,-1,0,-1,0,1,-1,-1,0,-1,1,1,1,0,1,0,-1,0,0,0,0,0,1,1,1,0,0,0,0,0,-1,0,-1,-1,-1,1,-1,0,0,0,-1,-1,0,0,-1,0,0,0,0,1,1,-1,0,0,-1,0,0,0,0,-1,0,1,0,0,0,-1,0,-1,-1,-1,-1,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,1,0,0,-1,-1,1,0,0,0,0,-1,0,1,0,0,-1,-1,1,0,0,-1,1,0,0,-1,0,1,1,-1,0,-1,1,1,-1,0,0,-1,0,-1,0,-1,1,0,-1,0,0,-1,-1,0,1,-1,0,1,1,0,0,0,-1,1,0,0,0,1,1,0,0,-1,0,1,1,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,1,0,0,-1,0,0,0,1,0,1,1,0,0,0,0,-1,1,1,0,0,-1,1,0,-1,0,1,1,0,-1,0,-1,0,0,0,1,-1,0,-1,0,1,0,1,-1,0,0,1,0,-1,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,-1,1,-1,-1,1,-1,1,0,1,0,0,1,-1,-1,-1,0,1,1,0,-1,-1,-1,0,0,-1,0,0,-1,0,-1,-1,-1,0,1,0,0,0,1,0,0,-1,0,0,0,1,-1,1,1,0,0,0,1,-1,0,0,-1,1,1,0,0,0,1,0,1,0,1,0,0,1,-1,0,0,0,0,1,-1,0,0,1,-1,-1,-1,0,1,1,0,0,0,-1,0,1,-1,-1,1,0,1,-1,-1,0,-1,0,0,1,0,-1,0,-1,0,1,-1,0,-1,1,0,0,0,0,0,-1,0,-1,-1,-1,1,1,-1,-1,-1,-1,1,0,1,0,0,-1,0,1,-1,0,1,0,0,-1,1,-1,0,-1,0,-1,0,-1,1,1,0,1,-1,0,0,1,-1,1,0,0,-1,0,0,0,0,1,0,0,-1,0,-1,1,0,0,0,0,0,-1,0,1,1,-1,-1,-1,0,1,1,0,0,0,0,1,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,-1,0,-1,1,1,-1,0,1,0,1,0,0,0,1,0,-1,-1,0,0,0,0,1,0,0,1,0,1,-1,-1,0,0,0,-1,0,0,0],[0,0,0,0,1,0,0,-1,1,1,1,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,-1,-1,1,1,0,0,0,-1,-1,-1,0,0,-1,0,-1,0,0,0,-1,-1,1,-1,0,0,0,0,0,-1,1,0,0,1,1,0,1,1,0,0,1,0,1,-1,0,0,-1,0,-1,1,0,1,0,1,0,1,0,0,1,-1,0,-1,1,0,0,-1,0,-1,1,0,0,0,0,1,0,0,0,-1,-1,1,-1,0,1,-1,-1,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,-1,0,1,0,0,0,1,0,0,-1,1,0,1,1,-1,1,0,0,-1,1,1,0,0,-1,0,0,0,0,1,1,-1,1,0,0,1,0,0,0,-1,0,0,0,-1,0,-1,1,0,0,0,-1,0,0,-1,-1,0,1,1,-1,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,-1,0,0,0,-1,0,0,1,0,0,-1,-1,0,0,0,-1,0,0,-1,0,0,0,-1,-1,-1,1,0,0,1,0,0,0,1,0,-1,0,1,1,-1,-1,0,1,0,-1,1,0,0,-1,0,0,1,-1,-1,0,0,1,0,1,0,-1,1,0,0,1,0,1,1,1,0,-1,0,0,1,0,1,1,1,-1,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,-1,-1,-1,0,0,-1,0,0,1,1,-1,-1,0,-1,-1,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,1,0,1,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,-1,1,1,0,-1,0,0,-1,-1,1,-1,0,-1,0,1,1,-1,0,0,-1,0,0,0,-1,1,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,-1,1,0,-1,1,1,0,0,0,-1,-1,0,0,0,0,-1,0,1,1,0,1,-1,-1,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,1,1,1,-1,-1,-1,0,0,0,1,-1,0,0,-1,1,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,-1,0,0,1,1,-1,0,0,1,0,-1,-1,-1,-1,-1,1,0,0,0,1,-1,0,-1,0,0,0,-1,0,1,-1,0,1,0,0],[0,0,0,-1,0,0,1,1,1,1,-1,0,-1,1,0,0,-1,1,-1,0,0,1,0,1,0,-1,0,-1,1,0,0,0,-1,0,0,1,0,0,1,0,0,0,-1,0,0,-1,0,1,0,-1,0,1,0,1,1,0,0,1,0,0,-1,-1,1,0,0,0,1,0,0,0,1,0,0,-1,0,-1,0,0,1,0,0,0,0,1,1,-1,0,0,1,-1,-1,1,1,0,1,0,0,1,0,0,0,0,0,-1,1,0,0,-1,0,0,0,-1,0,0,0,-1,1,1,0,0,1,0,1,0,0,-1,0,-1,-1,1,-1,0,-1,0,-1,-1,0,0,-1,0,1,1,-1,1,-1,1,1,0,-1,0,1,1,-1,0,1,1,-1,-1,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,1,-1,1,0,1,0,0,0,0,-1,0,0,1,-1,-1,0,-1,0,-1,1,-1,0,0,0,-1,-1,0,0,-1,0,0,0,0,-1,1,0,-1,0,0,1,-1,-1,0,0,1,0,0,-1,0,0,-1,1,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,1,1,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,1,1,1,0,1,-1,0,0,0,0,0,-1,0,1,-1,1,-1,1,-1,0,1,1,1,0,0,0,0,-1,1,0,-1,0,1,0,0,-1,1,0,0,0,0,-1,0,1,1,-1,0,-1,0,0,0,0,0,1,0,0,0,-1,-1,0,0,-1,0,1,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,-1,1,-1,1,0,0,-1,0,0,0,-1,0,0,0,1,1,0,0,1,1,0,-1,0,0,1,-1,0,1,1,1,0,-1,-1,1,0,1,-1,0,-1,0,0,0,-1,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,-1,-1,0,1,0,0,0,0,1,-1,0,0,1,-1,0,0,1,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,1,-1,0,0,0,-1,-1,-1,0,0,1,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,-1,0,1,0,0,0,1,1,1,0,0,0,1,1,0,0,0,-1,0,1,-1,0,0,0,0,1,1,0,0,-1,0,0],[0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,1,0,-1,-1,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,-1,0,-1,0,0,1,-1,0,0,0,1,0,1,-1,1,0,0,-1,0,-1,-1,1,1,-1,1,1,-1,1,0,0,-1,0,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,-1,0,1,-1,0,0,1,0,-1,-1,-1,0,1,1,1,0,-1,-1,0,0,1,-1,1,1,0,-1,1,-1,0,0,0,-1,1,0,0,0,0,-1,0,-1,0,-1,0,-1,1,0,0,1,-1,0,0,0,1,1,1,1,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,1,0,1,1,0,0,0,0,-1,1,-1,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,-1,-1,1,0,0,0,-1,0,1,-1,-1,1,0,0,1,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,1,0,1,1,1,0,0,-1,1,1,0,-1,0,0,1,-1,0,0,0,0,1,0,1,-1,0,-1,-1,0,0,0,1,1,0,-1,1,0,0,1,0,0,-1,1,1,0,0,1,0,1,0,-1,1,1,0,-1,0,0,1,0,0,-1,0,0,1,-1,1,0,0,-1,-1,0,0,-1,0,-1,0,0,0,0,0,0,1,1,0,-1,0,0,0,1,1,0,1,0,-1,0,0,-1,-1,0,0,1,1,-1,1,-1,-1,-1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,1,0,0,1,1,0,-1,1,0,1,-1,0,-1,1,0,0,0,0,-1,-1,0,-1,0,0,0,1,-1,0,-1,1,0,1,0,1,1,0,0,0,0,0,0,-1,0,0,0,-1,-1,1,-1,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,-1,-1,0,-1,0,-1,-1,0,1,0,0,-1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,-1,-1,0,0,-1,-1,0,0,1,-1,0,0,-1,-1,0,1,0,1,0,1,0,0,0,0,0,-1,1,0,0,-1,0,1,-1,0,-1,0,0,-1,0,1,0,0,0,1,0,-1,1,0],[0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,-1,0,0,0,1,-1,0,0,1,1,-1,1,-1,1,-1,1,0,-1,1,0,0,0,0,-1,0,-1,-1,1,-1,0,1,-1,-1,0,1,0,1,-1,0,1,0,0,1,1,1,1,-1,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,-1,1,1,-1,0,-1,-1,1,0,1,0,-1,0,1,-1,1,0,-1,0,0,1,0,1,0,-1,0,-1,-1,0,0,-1,-1,-1,-1,-1,1,0,1,0,0,0,0,-1,-1,-1,0,-1,1,1,0,0,-1,0,0,-1,0,0,0,-1,1,0,1,1,1,0,0,0,0,1,0,0,0,1,1,-1,-1,0,0,0,1,-1,-1,0,1,0,1,0,0,-1,1,0,0,0,0,1,1,-1,0,1,0,-1,0,-1,0,-1,0,0,0,-1,1,1,1,-1,1,-1,0,-1,1,0,1,0,-1,1,-1,1,-1,-1,1,0,-1,0,-1,-1,0,-1,1,0,-1,0,-1,0,0,0,1,0,0,1,-1,-1,1,0,-1,-1,0,0,0,-1,-1,0,0,1,0,0,0,0,1,0,-1,-1,-1,1,0,0,1,0,-1,-1,1,0,0,0,0,1,1,1,1,0,1,0,0,0,0,0,1,1,0,-1,-1,0,0,-1,0,0,-1,-1,0,-1,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,1,-1,-1,1,1,1,0,0,-1,-1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,-1,0,0,-1,1,1,0,0,0,1,-1,1,-1,-1,0,0,-1,0,0,-1,0,1,0,-1,-1,1,-1,0,-1,0,0,0,1,1,0,1,0,0,1,0,-1,1,1,0,-1,0,1,0,0,-1,-1,0,0,1,0,0,0,0,1,0,0,0,0,-1,-1,-1,1,1,0,0,1,-1,1,0,-1,0,0,1,1,0,1,0,1,1,0,1,0,1,0,0,0,1,1,1,-1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,-1,1,1,0,-1,0,-1,0,-1,1,0,1,1,0,0,0,0,0,1,-1,-1,-1,1,-1,-1,0,0,1,0,1,0,1,1,-1,-1,0,0,0,1,0,0,0,1,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,1,0,0,0,0,-1,0,1,0,1,0,1,1,0,0,0,0,0,0,0,-1,1,0,0,1,1,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,-1,0,-1,0],[0,-1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,-1,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,-1,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1,-1,1,0,0,1,0,0,1,0,1,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,-1,-1,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,-1,0,0,0,0,1,0,1,0,-1,0,-1,-1,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,-1,0,0,0,0,0,-1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,1,0,-1,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,-1,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,0,-1,-1,0,0,-1,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0],[0,1,0,0,0,-1,1,-1,0,1,1,0,0,0,0,-1,0,1,0,0,0,-1,-1,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,1,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,-1,0,-1,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,1,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0],[0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]]
weights['h2']=[[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,1,-1,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0],[0,1,0,0,-1,1,1,-1,0,-1,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,-1,-1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,1,0,-1,0,0,1,0,0,-1,0,0,0,0,1,1,-1,0,1,0,0,0,1,-1,0,1,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,-1,0,0,-1,0,0,1,0,0,0,-1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,1,1,0,0,0,-1,0,-1,0,1,1,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,1,0,1,0,0,0,-1,0,0,-1,-1,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0],[0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,1,0,-1,0,1,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,-1,-1,0,0,0,0,0,1,0,1,0,0,0,-1],[0,0,0,0,-1,0,0,0,1,-1,0,-1,0,0,-1,-1,0,0,-1,0,0,-1,0,0,0,0,1,1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,-1,-1,0,0,0,0,0,-1,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,0,0,1,0,1,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,1,-1,-1,-1,0,0,-1,1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,-1,0,1,0],[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,1,1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,-1,-1,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,1,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,1,0,0],[-1,0,-1,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0],[0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,-1,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,1,-1,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,-1,-1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,-1,1,0,0,1,0,0,0,-1,0,0,1,-1,-1,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,1,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,-1,0],[0,1,0,0,0,-1,1,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,-1,0,0,1,0,0,-1,0,0,1,0,0,0,0,-1,1,0,-1,1,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,-1,-1,0,0,1,0,0,0,1,0,0,0,0,1,-1,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,-1,0,0,0,-1,1,0,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0],[1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,-1,-1,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0],[1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,1,-1,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,-1,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,1,0,-1,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,-1,-1,0,-1,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,-1,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0],[0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,-1,0,0,1,0,0,0,1,0,-1,0,1,1,-1,1,0,0,0,0,0,0,0,1,-1,0,0,1,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,1,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,-1,0,0,-1,1,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,1,0,0,1,-1,1,-1,0,0,0,-1,0,1,0,0,1,-1,1,1,0,0,1,0,0,0,0,0,0,-1,0,-1,-1,1,-1,1,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,-1,-1,-1,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,-1,-1,0,0,0,1],[1,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,-1,1,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0],[0,-1,-1,-1,-1,0,0,-1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,1,0,0,-1,0,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,1,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,-1,1,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,-1,0,0,0,0,1,0,0,0,0,-1,1,-1,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,-1,0,0],[0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,-1,1,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,-1,0,1,-1,-1,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,1,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,1,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,1,0,0],[0,1,0,0,0,1,0,-1,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,1,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,1,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,1,1,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,1,0,-1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0],[0,1,0,-1,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,-1,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,-1,0,-1,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,0,-1,-1,0,0,0,0,0,0,-1,-1,0,0],[-1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,-1,1,-1,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,-1,-1,0,0,0,-1,0,-1,-1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,-1,-1,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,1,0,1,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],[0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,1,0,-1,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,1,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,-1,1,0,0,0,0,-1,0,0,1,0,-1,1,-1,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,-1,0,-1,0,-1,0,0,1,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,-1,-1,0,0,1,1,1,0,0,0,0,0,0,0,1,-1,1,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,-1,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,1,0,0,0,0,-1,0,1,0,1,0,-1,0,0,0,-1,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,-1,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,1,1,0,0,-1,-1,0,1,0,0,1,0,0,1,0,1,1,0,1,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,-1,-1,0,0,0,-1,1,0,0,-1,0,-1,0,0],[0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,-1,0,0,0,0,-1,0,0,0,0,-1,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1,0],[0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,1,0,-1,0,0,-1,0,1,0,1,0,-1,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,1,0,-1,0,1,1,0,1,0,0,0,0,0,-1,0,-1,-1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,-1,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0],[-1,0,1,0,-1,0,-1,0,0,0,-1,-1,0,1,0,0,1,-1,0,1,1,-1,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,-1,-1,0,-1,1,0,0,0,1,0,0,1,0,-1,0,0,1,-1,0,1,1,0,0,1,-1,0,1,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,1,0,0,1,0,1,-1,0,0,0,1,1,-1,-1,1,-1,-1,-1,0,0,-1,0,1,0,0,0,-1,0,-1,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,1,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,1,1,1,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,-1,1,-1,1,0,0,1,-1,0,0,0,1,-1,0,0,1,0,-1,1,0,1,-1,-1,1,1,-1,0,-1,-1,0,-1,-1,1,1,-1,0,0,0,0,-1,0,1],[0,0,0,0,-1,0,0,0,1,0,0,1,0,-1,-1,0,-1,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,-1,1,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,1,-1,0,0,0,-1,1,0,0,0,0,1,0,1,-1,-1,0,0,0,1,0,1,-1,1,0,0,0,-1,0,0,0,-1,0,1,1,0,0,0,0,0,1,1,1,1,0,0,0,1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,1,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,-1,-1,0,0,-1,0,0,0,0,0,1,0,1,-1,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0],[0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,1,0,1,0,0,0],[0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,1,0,1,0,-1,0,1,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,1,1,0,0,-1,0,-1,0,0,0,1,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,-1,-1,0,-1,-1,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,1,0,1,0,0,0,0,1,0,0,-1,0,1,0,0,0,1,0,0,0,1,0,-1,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,-1,-1,0,1,0,0,0,0,0,-1,-1,0,0,0,1,1,0,0,0,0,-1,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,1],[1,-1,0,0,0,0,0,-1,1,0,-1,0,1,0,0,0,-1,0,-1,-1,0,-1,0,-1,-1,-1,0,1,0,-1,-1,0,-1,-1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,-1,0,0,1,0,0,1,1,-1,0,0,0,0,0,0,1,0,1,0,0,-1,0,-1,0,-1,0,0,0,0,1,1,0,-1,0,0,0,0,0,1,0,0,0,0,-1,-1,-1,-1,1,1,0,0,-1,0,0,0,-1,1,-1,1,0,0,0,0,0,0,1,0,0,-1,1,0,0,-1,0,-1,0,-1,-1,1,1,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,-1,-1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,-1,0,0,1,0,1,1,1,0,1,1,-1,-1,0,0,-1,1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,1,0,1,-1,0,-1,0,0,0,0,-1,-1,0,0,0,1,0,0],[-1,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,-1,-1,0,0,0,0,1,0,0,0,1,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0],[-1,-1,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,1,1,0,0,1,0,0,0,0],[0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,1,0,0,1,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,-1,0,1,0,-1,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,-1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,-1,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,1,1,0,0,0,-1,1,1,-1,0,1,-1,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,0,-1,-1,-1,0,1,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,1,0,0,0,1,0,1,-1,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,-1,1],[0,0,1,-1,1,0,0,-1,-1,0,0,0,-1,1,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,-1,0,1,1,-1,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,-1,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0],[0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,-1,0,-1,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,1,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,1,-1,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,-1,0,-1,0,-1,0,0,0,0,-1],[0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,-1,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,-1,-1,-1,0,0,0,0,-1,0,-1,0,-1,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,1,1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,1,0,0,0,1,0,0,-1,0,1,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0],[0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,1,0,-1,0,0,-1,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,-1,0,1,0,0,0,0,-1,0,0,0,0,1,1,0,0,1,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,-1,1,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,-1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1],[0,-1,-1,1,0,0,0,1,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,-1,0,0,1,-1,1,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,-1,-1,0,0,0,-1,0,1,0,1,-1,0,-1,-1,1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,1,0,1,1,1,-1,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,1,-1,-1,0,0,-1,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,-1],[0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,1,0,0,0,0,-1,0,0,-1,1,0,0,0,0,1,1,1,0,-1,0,0,-1,0,1,0,0,-1,0,-1,-1,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,-1,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,1,-1,0,0,-1,1,0,0,0,-1,0,1,0,1,1,1,0,-1,-1,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,-1,0,-1,0,0,0],[0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,1,0,1,-1,1,0,0,1,-1,0,0,0,0,-1,0,0,-1,-1,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,-1,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,1,0,-1,-1,1,0,1,0,-1,0,0,1,0,-1,0,0,1,-1,-1,1,-1,-1,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,1,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,-1,-1,-1,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,1],[0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,1,-1,1,1,0,0,0,-1,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,-1,1,0,1,1,0,-1,0,-1,0,0,0,0,0,-1,0,1,-1,-1,0,0,-1,0,1,0,1,1,0,1,0,-1,0,-1,0,0,1,0,0,-1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,1,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,1,1,-1,0,0,1,0,0,0,-1,-1,-1,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,-1,1,0,0,0,1,0,0,0,1,-1],[0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,-1,0,1,1,-1,1,1,0,0,-1,0,0,0,-1,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,1,0,1,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,-1,1,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,1,0,0,0,-1,0,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,1,0,-1,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,-1,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,1,0,-1,0,-1,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,-1,0,1,1,0,1,1,1,0,-1,0,1,0,1,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,-1,1,1,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,-1,0,0,0,1,1,-1,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,-1,1,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,1,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,1,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,-1,0,0,-1,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,1,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,-1,1,-1,0,0,1,0,-1,-1,0,0,1,0,-1,0,-1,0,0,0,0,0,0,0,1,-1,0,1,1,0,0,0,1,1,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,1,-1,0,-1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,1,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,-1,1,1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,-1,0,-1,-1,0,0,-1,0,0,0,-1,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,-1],[1,0,0,0,0,0,1,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,-1,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,1,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,-1,-1,-1,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,0,0,-1,1,0,0,1,0,0,0,-1,0,-1,1,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,1,-1,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,1,-1,1,0,0,0,0,1,-1,-1,0,-1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,1,-1,1,0,0,0,0,1,1,0,1,1,0,0,0,0,0,-1,-1,0,0,0,1,0,1,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,-1,0,-1,0,-1,0,-1,0,-1,0,-1,1,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0],[0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,-1,0,1,0,0,0,0,0,-1,1,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,-1,0,1,0,0,0,0,0,0,-1],[0,-1,0,-1,0,0,0,0,0,0,-1,0,-1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,-1,-1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,-1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0],[0,0,-1,0,0,0,0,-1,0,1,0,-1,0,0,0,-1,1,0,0,0,-1,0,-1,1,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,-1,0,-1,-1,0,0,0,-1,0,0,0,1,-1,1,0,-1,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,-1,0,0,0,0,1,0,0,0,-1,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0],[0,0,-1,0,0,0,1,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,-1,0,-1,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,-1,1,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,1,1,0,-1,0,0,0,-1,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0],[-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,1,-1,0,0,0,-1,0,-1,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,-1,-1,0,0,1,0,1,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,1,-1,0,0,0,1,0,1,0,0,0,1,0,-1,1,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,1,-1,0,0,-1,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,-1,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,1,0],[-1,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,-1,0,0,0,-1,1,1,-1,0,1,0,0,0,1,0,0,-1,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,1,1,0,1,0,0,-1,0,1,1,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,1,-1,1,1,0,0,0,-1,0,0,0,-1,-1,0,0,0,1,0,1,0,0,1,0,0],[0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,-1,0,0,0,0,1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,-1,-1,1,-1,1,0,1,0,0,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,1,1,-1,0,0,0,-1,-1,-1,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,-1,1,0,-1,0,-1,1,-1,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,-1,-1,-1,0,-1,0,-1,0,0,0,-1,0,0,0,0,-1,1,1,-1,1,0,0,-1,1,0,0,0,-1,0,0,0,0,0,-1,1,0,1,0,0,0,0,1,0,1,-1,0],[0,0,0,-1,1,0,0,-1,0,0,1,-1,0,0,-1,0,0,-1,0,0,1,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,1,-1,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,1,0,-1,0,0,0,1,0,-1,0,0,1,0,0,0,-1,-1,-1,0,0,0,0,1,0,0,1,0,-1,0,1,-1,0,1,-1,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,1,-1,1,1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,1,0,-1,1,1,0,0,0,0,-1,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,-1,0,-1,0,1,0,0,0,0,-1,-1,0,0,1,1,0,0,0],[0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,-1,-1,0,-1,0,0,0,0,0,0,1,1,0,0,-1,-1,0,-1,0,0,-1,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,1,-1,-1,0,0,1,0,0,-1,-1,0,-1,0,0,1,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,-1,-1,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,1,0,0,1,0,0,1,0,0,0,0,1,0,-1,0,-1,0,1,0,0,1,0,-1,0,-1,0,1,1,-1,0,0,0,-1,0,-1,1,-1,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0],[0,0,0,0,1,0,0,1,0,-1,0,-1,-1,1,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,-1,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,0,-1,-1,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],[-1,0,-1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,-1,0,1,1,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,-1,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,1,0,0,-1,0,0,-1,0,0,-1,0,1,0,0,-1,0,0,-1,0,-1,0,0,0,-1,-1,0,0,0,0,1,1,-1,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1],[0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,1,-1,0,0,-1,0,-1,1,1,0,1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,-1,0,0,-1,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0],[0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,1,1,0,-1,0,-1,0,1,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,-1,0,1,-1,0,0,0,0,0,0,-1,0,1,0,1,1,0,1,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,-1,0,0,1,0,0,0,1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,1,0,0,-1,1,0,0,0,-1,-1,0,0,0,0,0,0],[0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,-1,-1,0,0,0,0,1,0,1,0,-1,0,0,0,1,0,0,-1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,1,-1,0,0,0,0,0,-1,0,0,1,1,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,1,0,0,0,-1,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,-1,0,0,1,-1,0,1,-1,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,1,0,0,-1,0,0,-1,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0],[-1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,1,-1,0,0,0,0,1,0,0,1,0,-1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,1,0,1,0,1,1,1,0,-1,-1,0,0,-1,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,-1,-1,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,-1,1,0,-1,1,0,0,0,0,0,0,1,-1,-1,0,1,1,1,0,0,-1,-1,0,0,0,1,-1,0,0,0,0,0,-1],[0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,-1,0,1,0,0,-1,-1,-1,1,-1,0,0,0,0,0,1,0,0,-1,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,1,-1,0,0,-1,0,0,-1,-1,1,-1,1,1,0,-1,0,0,0,0,1,0,0,1,1,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,-1,-1,1,1,0,-1,1,0,0,0,0,0,0,-1,0,1,-1,0,0,-1,0,0,1,-1,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,0,-1,0,1,1,0,0,1,0,0,0,0,1],[0,0,-1,0,0,1,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,1,-1,0,-1,0,-1,-1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0],[-1,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,1,0,0,1,0,0,0,0,0,1,-1,-1,0,0,1,0,-1,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,-1,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0],[0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,1,0,0,0,1,-1,1,0,1,1,0,1,1,0,1,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1],[1,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,1,0,1,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,-1,0,0,-1,1,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,-1,0,-1,0,0,-1,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,-1],[0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,-1,0,1,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,1,1,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,-1,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,-1,-1,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,1,1,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,1,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,1,0,0,0,1,0,0,0,0,-1,1,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,1,0,-1,0,-1,0,-1,1,0,1,0,-1,0,1,1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,-1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,-1,-1,0,-1,0,0,0,-1,0,-1,0,0,1,0,-1,0,0,1,0,1,0,-1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0],[0,1,1,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,-1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,-1,1,0,0,1,-1,0,0,-1,0,0,-1,-1,0,0,0,0,-1,0,-1,0,0,0,1,0,-1,0,0,0,-1,0,-1,0,1,0,-1,1,1,1,0,0,0,0,1,0,1,0,0,-1,-1,0,-1,0,0,0,0,0,0,-1,0,0,1,1,1,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,1,0,1,-1,0,0,1,0,0,-1,0,0,1,1,0,1,-1,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,1,1,0,0,0,-1,0,0,0,0,0,0,0,1,-1,1,-1,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,1,1,0,0,0,0,1,1],[0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,-1,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,-1,0,-1,1,0,0,-1,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,1,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,1,0,0,-1,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0],[0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,1,0,0,-1,1,1,0,1,0,0,0,0,0,1,1,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1],[1,1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,-1,0,0,0,1,-1,0,0,-1,-1,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,-1,0,1,1,0,0,0,0,0,0,1,-1,0,0,1,1,-1,-1,0,0,1,0,0,0,0,-1,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1],[1,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,-1,1,-1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,-1,1,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,-1,0,0,1,0,-1,0,1,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,-1,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,1,0,-1,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,-1,0,-1,0,1,0,0,-1,0,0,0,1,0,-1,0,0,0,-1,0,0,0],[-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,-1,1,0,0,0,0,-1,1,1,0,0,-1,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,1,1,-1,0,0,-1,1,0,0,-1,0,0,-1,-1,0,1,0,0,-1,0,0,0,0,0,1,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,-1,0,0,0,1,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,1,0,1,-1,0,0,0,0,0,1,-1,0,0,0,-1,-1,0,0,0,1,-1,0,1,1,1,0],[1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,-1,-1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,-1,0,0,0,1,0,-1,-1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,-1,-1,0,-1,0,1,0,-1,1,0,1,0,0,1,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,-1,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,-1,0,0,1,1,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,1,0,1,-1,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,1,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,1,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,1],[-1,-1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,-1,1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,0,1,-1,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,1,1,0,0,0,0,1,0,-1,1,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,-1,0,0,1,1,0,0,0,0,0,0,-1,-1],[0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,1,0,1,0],[1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,1,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,-1,0,0,-1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,1,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0],[-1,1,0,0,0,0,1,0,1,-1,0,0,1,0,0,1,0,0,0,1,0,-1,0,1,0,0,0,-1,1,0,-1,1,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,1,0,0,1,0,0,-1,0,0,0,0,0,1,0,-1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,-1,-1,0,0,-1,1,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,-1,0,0,-1,-1,0,1,1,1,-1,0,0,0,0,1,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,1,-1,-1,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,1,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,1,0,1,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,-1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],[0,-1,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,1,1,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,-1,1,0,0,1,0,0,0,0,-1,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,1,0,0],[0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,-1,0,-1,0,0,0,1,-1,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,1,-1,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,-1,0,0,0,0,0,-1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,-1,0,0,-1],[0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,-1,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,1,-1,-1,1,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1],[0,0,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,1,-1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,-1,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,1,1,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,-1,0,-1,1,0,-1,0,0,0,0,0,0,1,1,0,-1,0,0,-1,0,-1,-1],[0,1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,-1,-1,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,-1,-1,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,1,-1,0,0,-1,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,1,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,-1,-1,0,1,0,-1,0,0,0,0,0,0,0,-1,0,1,-1,0,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0],[0,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,0,0,-1,-1,-1,0,0,-1,0,0,-1,0,1,0,0,-1,0,1,0,0,0,0,1,1,0,0,0,-1,0,-1,0,1,-1,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,1,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,1,1,1,0,0,0,0,0,-1,-1,-1,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,1,0,0,0,-1,-1,0,0,0,0,0,-1],[0,-1,1,-1,-1,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,-1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,-1,1,0,-1,0,0,0,0,1,0,1,-1,0,0,1,0,0,1,0,1,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,-1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,-1,1,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,1,0],[-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,1,0,-1,0,0,0,0,0,0,-1,1,-1,1,0,-1,0,0,0,0,-1,0,1,0,1,0,-1,0,0,0,0,1,1,0,0,-1,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,-1,0,1,0,0,1,-1,-1,0,0,0,0,-1,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,-1,-1,0,0],[0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],[1,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,1,0,0,0,1,1,0,0,-1,0,0,0,1,0,0,0,0,0,-1],[0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,1,0,-1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,-1,0],[1,0,0,0,-1,-1,0,0,0,-1,1,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,-1,0,0,1,0,1,1,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,1,0,0,0,-1,1,0,0,0,1,-1,0,1,-1,0,0,0,0,0,0,1,1,-1,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,-1,0,0,0,0,0,-1,0,-1,-1,0,0,0,1,0,-1,0,1,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,1,0,-1,-1,0,0,0,1,-1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0],[0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,1,0,0,0,0,-1,-1,0,-1,0,1,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,-1,0,0,0,1,0,-1,1,0,0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,1,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,-1,0,0,1,0,0,0,0,1,1,-1,-1,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,-1,0,1,0,0,0,1,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,1,0,0],[0,0,0,0,-1,1,-1,1,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,-1,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,-1,1,-1,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,0,0,-1,-1,0,1,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,-1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,1,0,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,-1,0,1,-1,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,1,-1,0,0,-1,0,-1,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,-1,-1,-1],[0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,0,0,1,0,0,0,0,1,-1,0,0,0,-1,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,1,0,0,0,1,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,-1,1,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,1,1,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0],[0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,1,-1,-1,0,0,0,0,0,0,0,1,-1,1,-1,0,0,-1,1,-1,1,0,0,-1,0,0,-1,0,-1,-1,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,0,1,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,-1,1,0,1,0,0,0,0,0,-1,0,-1,-1,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,-1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,-1,1,1,1,0,1,0,-1,0,0,0,0,0,1,1,1,0,1,0,0,0,-1,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0],[-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,-1,1,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,-1,1,1,0,0,0,0,1,1,-1,0,0,0,0,0,1,0,0,0,1,0,0,-1,0,-1,-1,0,0,0,1,0,-1,-1,0,0,1,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0],[0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,-1,0,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,1,0,0,1,-1,0,0,1,0,-1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,-1,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,1,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1],[1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,1,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,0,0,-1,0,1,1,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],[-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,-1,0,0,0,0,0,0,-1,0,0,-1,-1,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0],[0,0,0,1,-1,-1,0,1,0,-1,0,-1,0,-1,0,0,1,0,-1,0,0,0,0,0,0,1,1,-1,-1,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,1,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,1,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,0,0,1,0,0,0,0,-1,0,-1,-1,0,0,0,1,0,0,0,0,-1,0,1,-1,0,1,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,1,0,0,0,-1],[0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,1,1,0,1,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,1,-1,1,0,0,0,0,1,1,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,1,0,0,0,-1,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,-1,0,1,1,-1,-1,0],[-1,0,1,0,1,-1,0,1,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,1,1,0,-1,0,1,1,0,1,-1,0,0,0,1,0,-1,0,1,1,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,1,0,0,0,1,-1,1,0,-1,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,-1,0,1,1,0,0,0,1,0,0,0,0,0,-1,0,1,0,-1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,1,0,1,-1,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,-1,0,1,0,1,-1,-1,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,0,-1,-1,0,0,0,0,-1,-1,0,0,0,0,0,-1,-1,0,1,0,1,0,0,0,0,0,0,-1,0,1,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0],[0,0,0,0,0,0,1,0,0,0,1,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,1,1,0,-1,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0],[0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],[1,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,-1,-1,0,-1,-1,1,1,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,-1,0,0,1,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,1,0,1,0,-1,0,-1,-1,1,0,0,-1,0,1,1,-1,-1,1,0,1,0,0,-1,0,-1,0,0,0,1,0,0,-1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0],[0,0,0,-1,1,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,1,1,0,1,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,-1,1,0,1,0,0,0,0,1,-1,0,-1,0,0,0,0,-1,1,1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,1],[0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,-1,0,1,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,1,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,-1,0],[0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,1,1,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,-1,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0],[0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,-1,1,0,1,0,0,1,1,0,0,1,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,1,-1,0,1,0,0,0,0,1,0,0,0,0,0,0],[0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,-1,-1,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,1,1,-1,1,0,0,0,0,1,0,-1,0,1,0,0,-1,0,1,0,0,0,-1,0,0,1,1,-1,-1,0,0,0,1,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,1,0,0,-1,0,0,-1,1,-1,0,-1,0,0,0,-1,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,1],[0,-1,0,-1,0,-1,0,0,0,0,-1,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,-1,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,1,1,0,0,1,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,-1,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,1,1,0,-1,0,0,-1,0,1,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,-1,0,0,1,-1,0,0,1,0,1,-1,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,1,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,1,0,0],[0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0],[0,1,0,1,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,-1,1,0,0,-1,1,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,-1,1,0,0,1,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0],[0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,-1,1,1,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,1,1,0,0,1,0,-1,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,1,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,-1,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,1,1,0],[0,-1,0,0,0,0,0,0,1,-1,0,0,-1,1,0,0,0,-1,-1,0,0,0,1,-1,1,1,0,0,0,0,1,0,0,0,0,-1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,-1,0,1,1,0,0,0,-1,-1,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0],[0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,-1,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,-1,-1,0,-1,0,0,0,1,1,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,-1,0,1,0,0,0,0,-1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,-1,1,0,0,1,1,1,0,-1,-1,0,0,-1,0,1,-1,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,-1,0,0,1,1,-1,0,0,1,1,1],[1,0,0,0,1,-1,1,-1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,-1,1,0,1,0,1,0,0,1,1,1,1,1,1,1,1,-1,-1,0,1,-1,0,0,-1,0,-1,-1,0,0,-1,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,-1,1,0,0,0,0,-1,1,1,-1,-1,0,0,0,-1,0,-1,1,1,0,0,0,0,-1,0,0,0,1,0,-1,0,-1,0,-1,0,0,-1,-1,0,0,0,0,-1,-1,0,0,-1,-1,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,-1,0,0,-1,1,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,1,-1,0,0,-1,0,0,0,0],[0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,1,-1,-1,-1,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,-1,0,0,0,0,-1,1,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,1,0,-1,0,-1,0,-1,0,0,0,0,0,-1,0,1,1,0,0,1],[0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,1,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,-1,0],[0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,1,-1,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,1,-1,0,0,1,1,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,1,1,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,1],[0,0,-1,0,0,1,0,-1,-1,0,1,1,-1,-1,0,0,0,1,-1,-1,0,0,1,-1,0,0,1,0,-1,1,1,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,-1,0,-1,-1,1,0,0,1,0,0,0,0,-1,1,0,0,0,0,1,1,-1,0,-1,0,0,-1,0,0,-1,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,-1,-1,1,1,1,0,1,-1,1,0,1,0,0,0,-1,0,1,0,0,0,-1,0,1,-1,0,1,0,0,0,0,-1,1,0,0,-1,-1,0,1,1,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,1,1,0,0,-1,0,0,0,0,0,0,0,-1,1,-1,-1,0,0,0,0,-1,-1,-1,0,0,1,0,0,0,0,0,1,0],[0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0,-1,1,0,0,-1,-1,0,1,-1,1,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,1,0,1,-1,1,0,1,0,0,0,0,1,1,0,0,0,-1,0,-1,0,1,-1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1],[1,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,-1,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,-1,0,0,0,0,0,0,-1,0,-1,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,-1,0,1,0,-1,0,-1,1,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,-1,0,-1,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,1,1,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,-1,0,-1,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,1,0,1,0,0,1,0,0,0,0,-1],[0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,-1,1,0,-1,1,0,1,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,1,-1,1,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,1,1,-1,1,1,0,0,0,0,0,0,0,1,0,0,-1,-1,1,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,1,0,-1,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,1,0,0,1,-1,-1,0,1,0],[0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,1,0,-1,0,0,1,1,-1,0,-1,-1,-1,0,0,0,0,-1,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,-1,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,-1,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,-1,1,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,1,-1,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,-1,0,0,0,1],[-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,1,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,-1,0,1,-1,0,0,0,0,0,1,0,1,1],[0,0,0,-1,0,1,1,0,-1,0,-1,0,0,0,0,0,0,1,0,-1,1,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,1,-1,-1,0,0,0,0,0,-1,-1,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,-1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,1,1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0],[0,1,0,0,1,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,1,1,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,1,0,-1,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,1,1,1,0,0,-1,1,0,0,0,-1,0,0,1,1,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,1,-1,0,-1,0,1,0],[0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,1,-1,0,0,0,1,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,-1,0,-1,0,0,0,1,0,1,-1,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,1,0,0,-1,0,1,-1,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,-1,0,0,1,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1,-1,-1,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,-1,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,1,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,-1],[0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,1,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,1,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,1,-1,-1,0,0,-1,1,0,0,0,0,1,0,0,0,1,0,-1,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,-1,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,-1,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,1,0],[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,-1,0,0,1,1,0,0,0,0,0,0,-1,0,-1],[-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,-1,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,1,1,0,0,0,-1,0,-1,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,1,-1,1,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,-1,-1,0,-1,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,-1,0,1,-1,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,-1,-1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,-1,1,0,1,1,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,1,1,0,0,0,0,0,-1,-1,-1,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,-1,0,-1,0,0,0,-1,-1,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,1,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0],[0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,1,-1,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,1,1,0,0,0,0,1,0],[0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,-1,-1,1,-1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,-1,-1,-1,1,0,-1,0,0,0,0,-1,1,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,-1,0,0,1,0,-1,0,0,0,0,-1,0,1,0,0],[0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,-1,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,1,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,1,0,0],[0,0,0,0,1,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,-1,-1,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,-1,1,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,0,-1,0,1,0,0,0,1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,-1,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,-1,-1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0],[1,1,0,0,1,0,-1,-1,-1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,-1,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,1,1,0,0,-1,-1,0,0,1,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1],[1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,-1,0,0,-1,0,-1,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,1,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,1,-1,0,-1,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0],[1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,1,1,0,-1,0,0,0,0,0,0,1,1,0,-1,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1],[0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,1,0,-1,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,-1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0],[0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,1,0,1,0,-1,0,0,0,1,0,1,1,0,0,0,0,1,1,-1,1,0,0,-1,-1,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,-1,1,0,0,0,0,1,0,0,0,1,1,0,-1,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,1,-1,0,0,0,0,1,1,0,-1,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,-1,1,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,-1,0,0,-1,-1,-1,0,0,1,0,1,0,0,1,0,0,-1,0,0,1,0,0,0,0,-1,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1],[0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,-1,1,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1],[0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,1,0,-1,-1,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,1,0,0,0,0,-1,0,0,1,0,0,1,0,-1,-1,1,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0],[0,0,0,0,-1,1,0,-1,0,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,-1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,-1,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,1,0,0,0,1,0,0,0,-1,1,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0],[0,-1,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0,-1,1,0,0,0,1,1,0,-1,0,-1,0,0,0,0,0,1,1,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,-1,0,0],[0,0,-1,0,1,0,-1,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0],[1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0],[0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,0,1,0,-1,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,1,0,-1,0,0,-1,-1,0,1,0,0,1,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,1,1,-1,0,1,0,0,-1,-1,0,1,0,-1,0,0,0,0,0,0],[1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,1,0,0,0,0,1,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,1,0,0,0,1,0,-1,0,0,0,0,1,-1,0,-1,0,0,1,-1,1,-1,-1,0,1,-1,1,0,0,-1,1,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,-1,0,0,0,0,0,1,-1,1,0,0,0,0,0,1,0,0,-1,0,-1,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,1,0,0,0],[0,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,1,1,0,0,1,0,-1,0,0,0,0,0,1,0,0,-1,-1,-1,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,-1,0,0,-1,-1,1,0,0,0,-1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,1,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,-1,-1,-1,0,-1,0,0,1,1,0,0,0,-1,0,0,-1],[0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,-1,1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0],[0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,-1,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,1,1,1,1,-1,-1,0,0,0,0,0,1,0,0,0,0,-1,1,0,1,0,-1,0,1,0,0,0,0,0,-1,1,-1,0,0,0,-1,1,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0],[0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,0,1,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,1,0,0,0,1,-1,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,1,1,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,-1,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,-1,-1,0,0,-1,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,-1,0,1,1,0],[-1,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,1,1,1,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,-1,1,1,1,0,0,0,0,1,0,0,1,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0],[-1,0,-1,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,-1,0,1,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,-1,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,1,0,-1],[1,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,1,-1,0,0,1,0,0,-1,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,-1,-1,0,0,1,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,-1,1,0,0,0,0,0,0,1,-1,0,1,-1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0],[0,1,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0],[-1,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,-1,0,0,0,1,1,1,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,1,1,0],[0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,-1,1,1,0,-1,0,1,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,1,-1,-1,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,-1,1,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,-1,0,0,-1,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,-1,0,0,0,0,1,0,0,0],[0,0,0,0,1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,-1,0,0,1,0,-1,-1,0,0,-1,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,-1,0,1,0,0,-1,-1,0,0,0,-1,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0],[-1,-1,0,1,0,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,-1,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,-1,-1,-1,0,-1,-1,0,1,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,1,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1],[0,1,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,1,-1,1,0,0,0,-1,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,-1,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,1,-1,0,0,1,0,-1,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,-1,0,-1,-1,0,0,-1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,-1,0,1,1,0,0,0,0,0,1,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,1,1,0,0,1,0,0,0,-1,0,-1,0,1,1,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,1,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,1,-1,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,-1,0,0,0,1,0,0,0,1,0,-1,-1,-1,0,-1,0,1,-1,0,-1,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,1,-1,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,-1,1,0,-1,-1,0,0,-1],[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0],[-1,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,1,0,1,1,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,1,0,-1,0,0,0,0,1,0,0,0,-1,1,0,-1,0,0,0,-1,0,0,0,-1,0,-1,-1,0,0,0,-1,0,0,-1,1,0,0,0,0,1,-1,0,0,0,1,0,1,0,-1,0,0,0,-1,1,1,1,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,-1,0,0,0,-1,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,1,0,0,1,1,-1,0,-1,-1,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,-1,0,1,0,-1,1,0,0,0,0,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,-1,1,0,0,0,1,-1,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,-1,1,-1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0],[-1,0,0,0,-1,0,0,0,0,-1,-1,-1,0,-1,1,0,0,1,0,0,0,0,0,1,0,-1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,1,1,0,0,0,0,-1,0,1,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,-1,1,1,1,1,-1,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,-1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,1,1,-1,-1,0,0,0,0,1,0,0,-1,0,0,1,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,-1,1,0,0,1,-1,-1,0,1,0,0,-1,0],[-1,0,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,-1,1,0,-1,-1,0,1,-1,1,1,1,0,0,0,0,0,1,-1,1,0,0,1,0,0,0,1,1,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,-1,-1,0,0,0,1,0,0,1,-1,1,0,1,0,0,0,1,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,1,0,1,0,0,0,0,-1,1,0,0,0,1,-1,1,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,-1,0,0,0,1,0,0,0],[0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,1,0,0,1,1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,-1,0,1,-1,1,1,0,0,-1,0,1,0,1,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,-1,0,0,-1,0,0,0,1,0,0,1,0,0,0,-1,-1,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,1,1,1,1,0,-1,1,0,0,-1,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,-1,0,0,0,-1,0,0,1,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0],[0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,1,1,0,0,0,0,0,-1,0,-1,1,-1,-1,-1,0,-1,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,-1,0,0,1,0,1,0,0,0,1,0,0,0,-1,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,-1,-1,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,-1,1,-1,0,0,0,0,-1,-1,0,0,0,-1,0,-1,1,1,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,-1,0,-1,1,0,0,0,0,1,-1,1,0,0,0,0],[-1,-1,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,-1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,-1,0,1,-1,0,-1,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,-1,-1,1,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,-1,1,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,-1,-1,0,1,-1,-1,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,1,0,0,0,-1,0,1,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,1,0,1,0,1,1,-1,0,1,-1,0,-1,0,-1,0,1,1,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,-1,0,1,0,0,0,0,0,-1,1,0,-1,0,0,-1,-1,0,0,0,0,-1,0,0,-1,0,0,-1,-1,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,-1,1,0,-1,-1,0,1,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,-1,0,0,0,-1,-1,-1,0,-1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,1,0,1,0,0,0,0,-1,0,1,0,0,0,0,1,0,-1,1,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,-1,1,0,0,-1,-1,-1,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,-1,-1,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,1,0,1,-1,-1,0,1,0,0,-1,0,0,0,0,-1,0,0,1,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,-1,-1,0,1,0,0,0,0,0,0,-1,0],[0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,-1,0,-1,0,1,-1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,1,0,0,0,-1,0,-1,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,-1,0,-1,0,0,0,0,0,-1,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,-1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,-1,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,-1,1,0,0,0,0,-1,0,0,0],[0,0,0,0,-1,0,1,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,1,-1,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,1,0,1,0,0,1,1,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,-1,0,1,1,0,-1,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,1,-1,0,1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0],[0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,-1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,-1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,1,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,-1,0,1,0,0,0,0,0,-1,0,-1,0,1,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,-1,1,-1,1,-1,0,-1,0,0,0,0],[-1,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,1,-1,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,-1,1,1,0,0,0,0,0,1,0,0,0,-1,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1],[-1,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0],[0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,1,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,-1,1,0,0,-1,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,-1,0,1,-1,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,-1,1,0,1,0,0,-1,-1,1,0,-1,0,0,0,-1,0],[1,0,0,-1,-1,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,-1,1,0,1,-1,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,1,0,1,0,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,-1,0,0,-1,0,0,-1,0,1,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,-1,-1,0,0,-1,1,1,0,0,0,1,0,0,0,0,1,-1,-1,0,0,0,0,0,0,0],[-1,0,0,1,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,-1,1,0,0,-1,-1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1],[0,-1,0,0,0,0,0,0,0,-1,0,1,-1,0,0,0,0,0,1,-1,0,0,0,-1,0,-1,1,0,0,1,0,0,0,1,-1,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,-1,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,-1,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,-1,-1,0,-1,1,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,-1,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0],[0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,-1,-1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,-1,0,0,-1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,1,-1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0],[-1,0,0,1,-1,0,0,0,0,0,-1,0,0,0,0,1,0,-1,-1,1,0,-1,1,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,-1,-1,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,-1,0,-1,1,1,-1,1,0,1,0,0,0,0,-1,0,-1,0,1,0,-1,0,0,0,0,1,0,0,1,-1,1,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,-1,0,1,1,0,-1,0,1,0,0,0,0],[-1,1,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,1,1,0,0,-1,1,0,0,1,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,-1,-1,0,1,0,-1,1,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,-1,-1,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0],[0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,-1,1,0,-1,0,0,0,0,0,-1,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,-1,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,1,-1,1,0,-1,0,1,-1,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,1,-1,1,1,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,-1,0],[0,0,1,0,0,-1,1,0,0,-1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,1,0,-1,-1,0,0,-1,0,0,0,1,0,-1,0,0,0,0,-1,1,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,-1,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,-1,-1,0,0,1,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,0,0,1,-1,0,0,0,0,1],[0,0,-1,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,-1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,-1,0,-1,-1,0,1,0,0,-1,0,0,1,0,1,-1,0,-1,0,0,0,0,0,0,-1,1,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,-1,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,1,0,-1,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,1,-1,-1,0,0,0,0,1,0,0,1,0,1,1,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,-1,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,-1,0,1,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,1,-1,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,-1,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0],[0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,-1,0,-1,0,0,-1,0,0,-1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,-1,-1,1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,-1,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,-1,0,1,-1,-1,1,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,1,0,0,-1,0,0,-1,-1,0,0,0,0,1,-1,1,0,0,0,0,-1,1,0,-1,-1,0,0,-1,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,-1,0,0,0,-1,0],[0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,1,0,-1,0,0,-1,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,-1,1,1,-1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,-1,1,0,0,1,1,0,0,0,0,1,1,0,0,-1,0,0,0,-1,1,0,-1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,-1,-1,0,0,0,0,0,1,1,-1,-1,0,0,0,1,0,0,-1,-1,0,0,1,0,0,1,-1,0,0,0,0,0,0],[-1,0,1,0,0,-1,-1,-1,0,1,0,0,0,0,0,0,-1,1,0,0,-1,1,0,0,0,0,-1,0,0,-1,1,0,0,-1,0,1,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,-1,0,0,1,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,-1,0,0,-1,-1,0,0,0,-1,1,1,0,0,1,0,1,-1,0,0,0,-1,-1,0,0,-1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,1,-1,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,1,1,1,0,0,0,0,0,1,0,0,1,0,-1,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,-1,1,0,-1,1,0,0,0,0,0,0],[0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0,1,0,1,0,-1,1,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,-1,0,-1,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,1,0,0,1,0,0,0,-1,0,1,0,1,0,0,1,0,0,-1,0,1,-1,-1,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,1,1,-1,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,1,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,0,0,-1,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,-1,1,0],[0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,-1,0,0,0,0,0,-1,-1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,-1,1,0,1,0,0,1,0,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,0],[0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,1,0,0,-1,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,1,1,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,1,1],[0,0,0,0,0,0,1,-1,-1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,1,0,0,0,-1,-1,-1,0,0,0,0,1,0,0,-1,1,0,0,-1,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,-1,-1,1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,-1,0,-1,-1,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1],[1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,-1,0,1,1,0,0,0,0,0,-1,0,0,0,1,1,1,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,-1,-1,0,-1,0,0,0,-1,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1],[0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,1,0,-1,-1,0,0,0,0,0,0,1,0,-1,0,1,-1,-1,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,-1,0,1,-1,-1,0,0,0,0,0,-1,0,0,-1,0,1,0,1,-1,0,0,0,0,0,1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,-1,1,0,-1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,1,1,-1,0,-1,0,1,1,-1,0,0,0,0,1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,-1,-1,1,0,-1,0,0,1,0,1,1,1,0,0,0,0,-1,0,0,1,0,0,0,1,-1,1,0,-1,0,0,0,-1,0,1,0,1,1,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,-1,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,-1,0,1,0,1,1,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,1,-1,0,0,0,0,0,-1,-1,0,0,0,1,0,0,-1,1,0,1,0,0,0,0,0,-1,0,0],[0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,-1,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,-1,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,-1,0,0,1,-1,-1,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0],[0,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,1,0,0,0,1,0,0,1,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,-1,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,0,-1,0,0,0,1,1,0,0,0,0,0,-1,0,-1,0,1,-1,-1,0,0,0,1,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,1,-1,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,0],[-1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,1,-1,0,-1,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,1,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,1,0,0,0,1,0,0,0,1,1,0,0,-1,1,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,-1,1,1,0,0,-1,0,0,-1,-1,0,-1,0,1,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,-1,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,-1,0,-1,1,-1,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,1,-1,-1,0,0,1,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,1,0,-1,0,-1,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,1,-1,0,0,-1,0,0,0,-1,-1,1,0,1,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,-1,-1,0,0,0,-1,-1,0],[0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,1,-1,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0],[0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,0,1,-1,0,1,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,-1,1,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,-1,-1,1,0,1,0,-1,0,1,-1,-1,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,-1,1,0,-1,0,1,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,-1,0,-1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,-1,0,-1,1,0,0,0,0,-1,0,0,1,1,1,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,-1,0,1,1,0,0,0,1,-1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,-1,0,0,0,-1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0],[0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,1,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,-1,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],[0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0],[-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,-1,1,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,1,-1,-1,0,0,1,0,1,1,0,0,0,1,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,-1,0,-1,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,-1,0,1,0,1,0,-1,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,-1,0,0,0],[1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,-1,0,0,-1,0,0,0,-1,-1,-1,1,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,-1,0,1,-1,0,0,0,1,0,0,-1,0,1,0,0,1,1,0,0,0,1,0,0,0,-1,0,0,1,0,1,0,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,-1,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,-1,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,-1,0,1,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,1,1,1,0,1,0,-1,1,1,-1,-1,-1,0,0,0,0,0,-1,0,0,0,1,1,0,0,1,0,0,-1,0,1,0,-1,0,-1,0,1,0,0,0,0,1,0,0,-1,1,1,0,0,0,0,0,0,0,1,-1,1,0,-1,1,-1,-1,0,0,0,0,0,-1,0,0,0,-1,0,-1,-1,1,0],[1,0,1,0,0,0,0,0,1,0,1,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,-1,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,1,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,1],[0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,1,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,-1,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,-1,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0],[0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,-1,0,1,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,1,0,0,1,0,0,1,0,0,0,-1,0,0,1,0,0,0,1,0,0,-1,-1,0,0,-1,1,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,1,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,1,0,0,1,0,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,0,0,-1,-1,0,1,-1,0,0,1,0,0,0,1,1,-1,-1,1,1,0,0,0,0,1,1,0,-1,0,1,0,1,0,0,1,0,-1,0,-1,1,1,0,1,-1,0,0,0,1,-1,0,0,0,1,0,-1,0,0,0,1,0,-1,0,0,0,0,1,1,1,1,-1,0,0,0,1,0,0,0,-1,1,1,1,1,1,1,0,1,0,-1,1,1,1,-1,0,-1,0,0,-1,0,1,-1,0,0,-1,0,1,0,0,-1,0,-1,0,0,-1,0,0,0,0,-1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,1,0,0,1,0,-1,-1,1,1,0,0,-1,0,-1,0,1,0,0,1,0,1,0,-1,1,1,0,0,0,1,0,-1,0,0,-1,1,0,0,-1,-1,-1,-1,1,0,0,0,0,0,1,-1,0,0,1,0,0,0,0,-1,-1,1,0,1,1,1,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,-1,0,-1,0,0,0,1,0,0,0,1,0,1,0,0,-1,1,1,0,0,0,-1,-1,-1,-1,-1,0,-1],[0,0,-1,0,0,-1,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,1,0,-1,-1,0,-1,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,1,-1,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,-1,0,-1,0,-1,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,-1,-1,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,1,1,0,0,1,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,-1,1,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,1,-1,1,1,0,0,0,1,0,0,-1,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,-1,1,0,0,0,0,0,-1,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,-1,-1,0,0,0,0,-1,0,0,-1,-1,-1,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0],[0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,-1,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,-1,0,1,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,1,0,-1,-1,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,1,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1],[0,0,0,0,0,0,1,0,-1,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,1,-1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,-1,-1,0,-1,0,0,0,-1,-1,-1,0,-1,0,0,0,-1,0,0,0,0,0,1,0,1,0,-1,1,-1,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,1,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,1,-1,1,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,-1,0,0,1,0,-1,0,0,1,1,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,1,0,0,0,-1,-1,0,0,0,0,-1,0,-1,1,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,-1],[0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,1,-1,0,0,1,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,1,-1,-1,-1,0,1,0,1,0,0,0,0,-1,1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0],[0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,-1,0,0,0,0,1,-1,0,0,0,0,1,0,0,-1,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,-1,0,1,1,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,0,1,0,0,0,0],[0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,1,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,1,1,-1,0,0,1,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,-1,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,-1,-1,0,1,-1,0,0,0,-1,-1,0,0,1,-1,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,-1,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,1,0,0,-1,0,1,-1,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,-1,0,0,-1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,1,-1,0,0,0,0,-1,0,0,0,1,-1,0,0,0,-1,0,0,0,0,1,1,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0],[-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,1,0,0,0,-1,0,0,1,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,-1,0,-1,-1,0,0,0,1,-1,0,0,1,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,-1,0,-1,1,-1,0,0,1,0,0,1,0,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,1,0],[-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,1,1,0,0,0,-1,-1,0,1,0,0,0,1,-1,0,0,1,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1],[1,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,-1,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,-1,1,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,-1,0,0,-1,1,1,1,1,0,0,0,0,0,-1,1,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,-1,-1,0,0,-1,0,-1,0,-1,0,-1,0,0,0,-1,0,0,0,1,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,-1,1,0,-1,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0],[0,0,1,0,1,-1,0,0,1,0,0,0,0,0,-1,-1,0,1,1,1,0,0,-1,-1,-1,0,1,0,-1,0,-1,0,1,1,1,1,1,-1,1,0,0,0,0,0,0,1,1,0,0,0,0,-1,1,0,1,0,0,0,0,0,-1,1,0,0,-1,0,-1,-1,0,0,0,-1,1,0,0,-1,-1,-1,1,0,-1,-1,0,-1,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,-1,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,1,1,1,-1,0,0,-1,0,1,0,-1,1,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,1,1,1,-1,-1,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,1,1,0,-1,0,-1,0,1,0,-1,0,1,1,1,0,0,0,1,0,-1,0,0,1,-1,1,-1,0,0,-1,0,-1,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,1,1,0,0,0,0,-1,0,0,0,-1,0,1,1,0,0,1,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,-1,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,1,0,1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,-1,0,0,0,1,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1],[0,0,-1,-1,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,-1,-1,0,-1,-1,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,-1,1,0,0,-1,0,0,-1,1,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,1,-1,0,0,0,-1,-1,0,0,1,-1,-1,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,1,0,1,0,0,0,0,0],[-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,1,1,-1,1,0,-1,-1,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,-1,-1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,1,-1,0,1,0,0,-1,0,0,-1,0,1,-1,0,1,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,1,-1,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,-1,-1,-1,0,0,0,1,0,0,0,0,0,0,-1,-1,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,1],[0,-1,0,1,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,1,-1,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,1,1,1,0,0,0,0,0,0,0,1,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,1,0,0,1,-1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,-1,-1,-1,0,0,0,-1,1,0,-1,0,0,1,0,0,0,0,-1,1,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,-1,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0],[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,1,0,-1,0,0,0,0,0,1,0,-1,-1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0],[-1,0,0,0,0,0,-1,0,-1,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,-1,1,0,-1,0,0,0,0,1,1,0,-1,1,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,1],[1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,1,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,-1,1,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,1,0,-1,-1,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,-1,-1,0,0,1],[0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,0,1,0,0,1,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,-1,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,-1,0,0,0,0,1,1,1,0],[0,0,1,-1,0,0,-1,0,0,0,-1,-1,-1,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,-1,0,0,-1,-1,1,-1,0,-1,1,1,0,0,-1,0,0,-1,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,1,0,-1,0,0,0,0,-1,1,1,1,0,0,-1,0,0,0,0,0,1,1,1,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,-1,1,0,0,0,-1,0,0,0,0,0,-1,1,1,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,-1,0,0,-1,0,0,0,0,0,0,-1,0,1,1,0,-1,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,-1,0,-1,1,-1,0,1,0,0,1,0,0,-1,0,-1,0,1,0,-1,0,0,0,0,0,0],[0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,-1,-1,0,0,1,0,0,1,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,1,-1,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1],[-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,-1,0,0,1,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,1,0,-1,1,-1,0,-1,0,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,-1,1,0,0,1,1,-1,0,0,1,0,-1,0,0,0,0,1,0,1,-1,1,-1,0,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,-1,-1,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,-1,1,1,0,0,1,0,1,0,0,1,0,1,-1,0,0,0,-1,0,0,0,-1,0,0,1,-1,-1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,1,0,0,0,0,1,0,0,0,1,-1,-1,1,0,-1,1,0,0,1,-1,0,0,0,0,0,-1,-1,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,-1,0,1,0,-1,0,0,0,-1,1,0,0,1,-1,0,0,0,-1,1,-1,-1,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,1,-1,1,1,0,0,0,1,0,0,0,1,0,0,-1,0,0,0,-1,0,0,1,0,1,-1,-1,-1,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,-1,0,-1,1,0,0,1,1,1,1,-1,-1,-1,1,1,0,-1,1,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0],[0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,1,0,0,0,0,1,0,0,1,0,0,1,-1,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,-1,0,1,0,0,-1,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,1,0,-1,0,1,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,-1,0],[0,1,0,1,1,-1,0,-1,-1,0,-1,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,-1,1,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,1,-1,0,0,-1,0,1,0,0,1,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,-1,1,0,0,0,0,0,1,0,0,1,1,-1,0,1,0,0,-1,1,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,-1,0,-1,-1,0,0,0,1,0,0,1,1,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,1,0,0,-1,0,-1,0,0,0,0,1,1,0,1,0,0,0,0,0,-1,1,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,1,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,-1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,-1,1,1,-1,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,-1,-1,1,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,1,-1,0,0,0,1,0,0,0,1,0,0,1,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0],[0,0,0,0,1,0,-1,-1,0,0,0,1,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,1,-1,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0],[0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,1,0,-1,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,1,0,-1,0,-1,1,0,-1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,-1,0,0,1,0,1,-1,-1,0,1,1,0,0,1,0,-1,0,-1,1,0,-1,0,0,0,0,0,-1],[-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,1,0,0,-1,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,1,-1,0,0,1,-1,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,-1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,-1],[0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,-1,0,1,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,0,0,-1,0,0,1,-1,0,-1,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,-1,-1,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1],[0,0,0,0,0,0,-1,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,1,0,1,0,0,-1,0,0,-1,-1,-1,0,-1,-1,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,-1,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,-1,-1,0,0,0,0,-1,1,0,-1,0,-1,1,0,-1,0,-1,0,1,1,0,0,1,1,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,-1,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,-1,0,-1,0,0,0],[0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,-1,-1,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,1,1,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,1,-1,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,-1,0,-1,1,0,0,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,-1,0,0,1,0,-1,-1,0,1,0,1,1,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,1,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,1,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1],[0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,-1,-1,0,0,-1,0,-1,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,-1,1,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,-1,-1,0,-1,-1,0,-1,0,0,0,1,1,0,1,0,0,-1,1,0,0,0,-1,0,0,0,0,-1,-1,-1,0,-1,0,1,0,0,1,-1,0,0,0,1,1,0,-1,0,0,0,-1,0,0,0,-1,0,1,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,-1,1,0,1,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1],[-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,-1,0,1,0,0,0,1,0,-1,-1,0,0,0,1,-1,0,0,0,0,0,-1,1,1,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,-1],[1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,1,-1,0,0,-1,0,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,-1,0,-1,1,-1,0,1,0,0,1,0,0,0,0,0,-1,-1,0,1,0,0,0,0,1,0,1,1,0,-1,0,1,0,-1,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,-1,-1,0,0,0,0,0,0,-1,-1,-1,-1,0,0,-1,0,-1,0,0,0,1,0,1,0,0,-1,-1,0,0,0,0,0,1,-1,0,1,1,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,1,-1,0,1,-1,0,0,-1,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,1,1,0,0,0,0,-1,0,-1,0,0,0,1,-1,0,0,0,0,1,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,-1,0,1,0,1,0],[1,1,0,-1,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,1,0,0,-1,0,1,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,1,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,-1,0,0,1,0,1,-1,0,-1,1,0,-1,0,0,0,-1,0,1,0,0,1,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,1,-1,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,-1,0,1,1,0,0,0,0,1,-1,0,-1,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,-1,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,1,0,-1,1,0,0,0,0,0,0,0,-1,0,1,-1,-1,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,1,0,0,1,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,-1,0],[0,-1,0,-1,-1,-1,0,0,-1,0,0,0,0,-1,-1,0,-1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,1,1,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,1,1,1,0,1,-1,-1,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,1,0,1,0,1,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1],[0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,1,-1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,1,1,0,-1,0,0,-1,-1,0,0,0,0,-1,-1,0,-1,1,0,-1,-1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,-1,-1,1,1,0,0,0,-1,-1,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,-1,0,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,-1,0,-1,0,0,-1,0,0,0,0,0,1,1,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,0,-1,0,0,0,0,-1,0,0,1,1,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,1,1,-1,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,1,0,0,1,0,1,0,1,1,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,1,0,-1,0,-1,0,0],[0,1,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,-1,0,1,0,-1,0,0,0,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,1,0,-1,0,1,0,1,0,0,0,0,-1,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,1],[0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,1,1,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,-1,-1,0,0],[-1,0,0,-1,-1,0,0,0,0,1,0,-1,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,-1,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,-1,-1,0,0,0,0,0,0,1,-1,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,-1,0,0,1,-1,-1,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,-1,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0],[0,0,0,1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,-1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,-1,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,1,0,0,-1,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,-1,0,0,-1,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,1,0,0,0,0,-1,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,1,0],[0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,-1,0,1,0,0,-1,0,0,0,1,-1,0,0,0,1,0,0,0,-1,0,-1,0,-1,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,-1,-1,0,-1,-1,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,-1,0,-1,0,0,0,0,-1,-1,1,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,-1,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0],[0,0,0,1,0,0,1,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,-1,0,-1,0,1,1,1,-1,1,0,0,0,0,0,0,0,0,1,0,1,0,-1,-1,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,1,0,0,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,1],[0,0,1,0,0,-1,0,0,1,0,1,0,-1,1,0,1,0,0,1,-1,0,1,0,0,0,0,0,0,0,1,-1,-1,0,-1,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,1,0,1,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,-1,0,1,0,-1,0,0,0,1,-1,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,-1,-1,0,0,0,-1,-1,1,0,1,0,0,1,-1,0,1,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,-1,0,0,-1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,-1,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,1,1,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,1,0,0,1,1,0,-1,0,0,0,1,1,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,-1,0,1,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0],[0,0,0,0,1,-1,0,0,0,0,1,0,0,-1,0,1,-1,0,0,-1,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,1,0,-1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,1,0,0,0,0,0,0,1,-1,0,-1,0],[1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,-1,0,1,0,0,0,-1,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,-1,0,0,0,1,0,0,0,1,0,1,0,0,-1,-1,0,-1,0,0,0,0,1,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,-1],[0,1,0,1,1,0,0,-1,-1,0,0,0,0,1,0,0,0,0,1,-1,1,0,0,0,1,0,0,-1,0,1,0,0,1,-1,0,0,0,0,0,0,1,0,-1,1,0,1,-1,0,0,0,0,1,0,0,0,1,-1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,1,0,0,0,0,0,1,1,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,-1,1,0,1,0,1,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,1,-1,0,0,1,-1,-1,0,-1,-1,0,0,-1,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,-1,1,0,0,0,-1,0,-1,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,-1,-1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0],[0,0,0,0,0,1,-1,0,0,-1,0,0,1,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,-1,1,-1,1,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,1,0,0,0,1,0,0,0,1,0,-1,0,-1,0,0,0,-1,1,0,1,0,-1,-1,0,0],[0,1,-1,-1,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0],[0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,-1,0,0,-1,-1,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,1,1,-1,0,0,1,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,-1,1,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,1,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,1,-1,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0],[0,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,-1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,-1,0,1,-1,-1,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,-1,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0],[0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,1,-1,0,0,-1,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,1,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,-1,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,-1,-1,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0],[0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,0,0,-1,-1,0,0,-1,0,0,0,1,-1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,-1,0,0,1,-1,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,1,-1,1,0,-1,1,0,0,0,-1,0,0,1,1,1,0,0,-1,0,-1,-1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,-1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,1,-1,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,1,0,0],[1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,1,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,1,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,-1,-1,0,0,-1,0,0,1,0],[0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,-1,0,0,0,-1,0,1,0,-1,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],[1,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,-1,0,-1,1,0,0,0,-1,0,1,0,0,-1,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,1,0,0,0,0,-1,1,-1,0,-1,0,1,0],[0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,-1,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,-1,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,-1,0,1,0,0,0,0,1,0,0,0,0,-1,-1,0,1,0,0,0,1,0,0,0,1,-1,1,0,0,0,1,0,0,0,0,0,-1,-1,0,-1,0,-1,0,1,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,-1],[0,1,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,1,-1,-1,1,-1,0,0,-1,-1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,-1,1,-1,0,0,1,0,1,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,-1,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,-1,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,-1,1,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0],[0,0,0,0,0,-1,0,-1,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,-1,0,0,1,1,0,0,-1,0,0,-1,-1,0,-1,0,0,0,0],[0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,1,1,0,0,0,0,-1,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0],[0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,0,1,1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0],[0,0,1,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,-1,-1,-1,0,0,0,0,0,-1,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,1,1,-1,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,1,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0],[0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,1,1,0,0,-1,-1,-1,0,0,0,0,-1,0,1,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,-1,-1,0,-1,-1,0,1,0,0,-1,-1,0,0,0,-1,-1,1,0,1,0,0,0,1,0,0,0,-1,0,0,-1,-1,0,1,1,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,-1,0,0,0,0,-1,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,-1,0,1,0,0,0,0,0,-1,-1,-1,-1,1,0,1,-1,1,1,0,0,0,0,-1,0,0,1,0,1,1,-1,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,1,-1,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,-1,0,1,-1,0,0,0,0,0,0,0,-1,-1,0,1,1,0,0,-1,0,0,0,1,0,-1,0,0,0,-1,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0],[-1,0,0,0,-1,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,1,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,-1,-1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,-1,0,0,1,0,-1,0,0,0,-1,0,1,-1,0,0,0,0,1,0,0,-1,0],[0,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,1,0,-1,-1,1,-1,0,0,0,0,1,0,0,1,-1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,0,1,-1,0,0,0,-1,-1,-1,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,1,-1,-1,0,0,1,-1,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,1,1,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,0,0,1,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,-1,1,0,-1,0,0,1,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,-1,0,0,1,-1,0,0],[0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,1,0,-1,0,0,1,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,1,0,0,1,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,1,0,-1,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,-1,1,-1,1,0,0,0,-1,-1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,-1,-1,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0],[1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,1,-1,0,0,1,0,0,0,0,0,1,0,-1,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,1,1,0,0,0,-1,0],[0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,-1,-1,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,-1,-1,0],[0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,-1,-1,-1,1,-1,-1,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,-1,0,-1,1,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,-1,1,0,0,-1,1,0,0,1,1,-1,-1,-1,1,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,1,-1,1,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,-1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0],[0,1,0,0,1,-1,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,1,0,-1,0,0,1,0,0,1,0,0,-1,-1,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0],[0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,1,0,0,-1,-1,0,-1,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1],[1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,1,0,-1,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,1,1,0,1,1,-1,1,0,1,1,-1,0,0,0,0,0,-1,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,-1,0,0,-1,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,1,1,0,0,0,0,0,-1,0,0,-1,0,0,1,-1,1,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,1,-1,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,-1,0,0,0,0,1,1,0,0,-1,1,0,1,0,0,0,0,0,1,0,1,0,-1,0,0,-1,0,0,1,0,0,1,0,0,0,1,-1,0,1,0,0,0,0,-1,0,0,0,0,0,-1,-1,1,-1,0,0,0,0,0,1,1,1,0,-1,0,0,0,0,-1,0,0,-1,1],[0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0],[1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,-1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,-1,0,0,1,0,-1,0,0,0,0,-1,1,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0],[0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,-1,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,-1,0,-1,-1,1,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,1,-1,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,1,-1,-1,0,0,1,0,0,0,0,0,1,1,1,0,1,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,1,-1,0,1,0,0,0,-1,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,1,1,0,0,0,1,0,1,1,0,1,0,0,0,1,0,0,-1,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,1,0,0,-1,1,0,0,1,0,0,-1,0,0,0,1,0,0,0,-1,0,1,-1,1,0,0,0,-1,1,0,-1,0,1,1,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,-1,-1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,-1,0,-1,0,0,0,0,1,0,1,0,-1,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,1,-1,-1,0,-1,0,0,1,1],[0,0,1,0,-1,-1,0,0,0,0,0,1,1,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,1,1,0,0,0,-1,0,-1,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,1,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,-1,0,0,-1,0,0,0,-1,0,1,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,1,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,1,0,1,0,0,1,0,0,1,0,0,0,1,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,-1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,1,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,1,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,1,1,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0],[0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,1,-1,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,-1,0,0,0,1,0,0,0,0,-1,-1,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,-1,0,-1,0,0,1,0,1,0,-1,0,-1,0,-1,0,1,-1,0,0,0,0,0,1,0,0,0,0,0],[0,0,1,-1,0,0,0,0,0,0,-1,1,0,0,1,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,-1,1,0,-1,0,0,0,1,1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,-1,0,-1,1,-1,0,1],[0,0,1,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,-1,0,-1,0,0,0,-1,-1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,-1,1,-1,0,0,1,1,0,0,0,0,1,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,-1,0,0,-1,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1],[0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,-1,0,1,1,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,1,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,-1,-1,0,0,1,-1,0,0,0,-1,0,0,-1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,1,-1,1,0,-1,-1,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,-1,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1],[0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,1,0,1,0,1,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,-1,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,-1],[0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,-1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,-1,-1,-1,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,-1,0,1,0,0,0,0,0,-1,1,1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1],[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,-1,-1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,-1,1,1,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,-1,1,0,0,1,0,0,0,0,1,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,1,0,0,0,0,0,0,1,1,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,-1,1],[0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,-1,0,1,-1,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,-1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,-1,0,0,-1,1,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,-1,0,0,0,0,0],[0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,1,1,0,-1,0,0,0,0,-1,0,1,-1,0,-1,0,0,0,0,0,0,1,0,0,1,0,1,0,-1,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,-1,0,0],[0,0,0,0,-1,0,1,1,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,1,-1,0,-1,0,0,-1,0,1,0,0,0,-1,1,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,-1,-1,0,1,0,0,0,-1,-1,0,0,-1,0,0,0,0,1,1,0,-1,0,0,0,0,1,-1,0,0,-1,-1,0,1,0,0,1,-1,0,-1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,-1,1,1,-1,0,-1,0,0,0,1,-1,1,0,0,1,-1,1,1,0,-1,-1,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,-1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,-1,0,1,0,0,0,1,-1,0,0,1,0,1,0,0,0,-1,0,1,0],[0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,1,0,0,-1,-1,1,-1,1,0,-1,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,1,0,1,0,0,0,-1,1,0,0,0,0,-1,1,0,0,-1,0,-1,0,0,0,0,1],[0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,-1,0,0,0,-1,-1,-1,1,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,1,0,0,1,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,1,0,0,0,0,0,-1,1,-1,0,0,-1,0,1,0,1,-1,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0],[0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,-1,0,-1,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,1,1,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,-1,0,1,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,-1,1,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,-1,-1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,-1,-1,0,-1,1,0,0,-1,-1,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1],[0,0,0,1,0,0,0,-1,1,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,-1,0,1,-1,0,1,0,-1,1,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,-1,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,-1,0,0,0,0],[0,0,1,-1,-1,0,0,-1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,1,0,0,0,-1,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,1,0,-1,1,0,-1,0,0,-1,0,1,0,0,0,0,-1,0,0,1,-1,-1,0,0,0,0,1,0,1,1,-1,-1,0,0,1,-1,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,1,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,1,-1,0,0,1,1,0,0,-1,0,0,0,-1,0,-1,0,0,1,0,0,1,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1],[0,0,0,0,-1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,1,1,0,-1,0,1,1,0,-1,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,-1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,1,0,1,0,-1,0,0,0,1,0,0,0,0,-1,0,0,-1,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,-1,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,1,0,0,0],[0,-1,-1,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,-1,0,0,0,0,-1,1,-1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,1,1,0,0,0,1,-1,1,0,-1,0,-1,0,0,0,0,1,0,0,0,0,1,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,-1,-1,0,0,0,0,1,1,0,0,0,0,-1,0,1,0,0,1,0,0,-1,1,1,0,-1,0,0,-1,0,0,0,0,0,1,0,-1,-1,0,0,-1,0,0,-1,1,0,1,0,0,1,1,-1,0,0,0,0,0,1,-1,0,1,0,0,0,-1,-1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,-1,1,-1,0,1,-1,0,0,-1,0,0,0,1,-1,-1,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0,-1,1,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,1,1,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,-1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,1,0,1,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,-1,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,-1,0,0,0,1,1,0,1,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1],[-1,1,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,-1,0,1,0,1,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,-1,1,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,1,-1,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1],[0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,-1,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,1,1,0,0,1,0,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,-1,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,1,0,0,0,0,1,0,1,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,-1,0,-1,0,1,0,-1,-1,1,0,1,-1,0,0,-1,1,0,0,0,0,1,0,0,0,-1,0,-1,0,1,0,1,0,0,-1,0,-1,1,0,0,0,-1,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,-1,0,1,-1,0,0,0,0,1,0,0,0,-1,0,0],[0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],[0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,1,0,0,1,0,-1,0,-1,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0],[0,-1,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,-1,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,-1,0,1,0,-1,0,0,-1,1,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,1,0,-1,0,0,0,1,0,1,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,1,1,0,0,0],[0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,-1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,-1,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,-1,-1,-1,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,-1,0,0,1,0,1,-1,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,-1,0,-1,-1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,1,0,0,0,-1,0,1,0,1,0,-1,0,0,-1,0,1,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,1,1,0,0,0,1,0,0,-1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1],[0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,-1,1,0,1,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,1,0,-1,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0],[-1,0,-1,0,0,1,1,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,1,-1,-1,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0],[0,0,0,0,1,-1,-1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,1,1,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,1,0,-1,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,-1,0,0,1,1,0,0,-1,1,0,0,-1,-1,0,0,0,0,0,-1,-1,1,-1,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0],[0,-1,-1,0,0,0,1,0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,1,0,-1,-1,1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,1,0,0,-1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,-1,0,1,-1,0,0,-1,-1,1,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,1,0,0,0,1,1,0,0,0,0,0,-1,0,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,1,1,-1,0,0,1,0,0,-1,0,0,-1,0,1,0,0,1,-1,0,0,0,0,-1,-1,0,1,-1,0,0,1,-1,0,-1,0,0,0,0,-1,0,0,-1,0,0,1,0,0,-1,0,0,-1,0,0,-1,-1,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,-1,-1,0,1],[1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,1,1,0,0,0,0,-1,-1,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,-1,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,-1,0,0],[0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,-1,0,0,-1,0,0,0,0,0,-1,1,0,0,-1,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,-1,-1,0,0,0,-1,1,1,0,0,-1,-1,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0],[0,1,-1,-1,-1,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,-1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,-1,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,1,0,0,1,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,-1,1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,1,-1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,-1,-1,0,-1,0,1,-1,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,-1,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,-1,-1,1,0,-1,0,-1,0,0,0,0,0,0,0,1,0,1,-1,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,-1,1,-1,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,1,1,0,0,0,0,1,1],[0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,1,0,1,0,-1,0,0,0,0,-1,0,-1,0,0,-1,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,-1,1,0,-1,0,1,0,0,1,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,1,-1,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,1,-1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1],[1,0,0,0,0,1,0,1,1,1,1,0,1,0,0,0,0,0,-1,-1,0,0,0,-1,1,-1,0,0,0,0,0,0,1,0,1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,1,-1,0,-1,0,0,0,0,-1,0,0,1,0,0,0,-1,-1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,-1,-1,0,0,0,0,1,0,0,0,1,1,-1,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,1,1,-1,0,-1,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,1,1,0,-1,0,0,0,0],[0,0,0,0,0,-1,1,0,0,1,0,0,0,1,0,-1,1,-1,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,1,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,-1,0,1,0,0,1,0,-1,1,0,0,0,1,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,-1,-1,0,0,0,1,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0],[0,1,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,-1,0,0,1,0,-1,-1,1,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,1,0,0,1,1,0,-1,0,0,1,-1,0,-1,0,0,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,1,0,-1,-1,0,0,0,1,0,0,0,-1,0,1,0,1,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,-1,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,0,0,0,1,-1,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,-1,0,1,1,0,0,-1,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,1,-1,0,-1,-1,0],[0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,0,1,0,1,0,-1,0,0,0,-1,0,0,-1,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,0,-1,0,0,1,0,0,0,0,1,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,-1,1,0,0,0,0,-1,-1,0,1,0,1,0,0,0,0,0,1,-1,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,-1,-1,0,1,1,-1,0,0,1,0,0,0,0,0,0],[1,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,-1,0,0,1,-1,0,0,0,0,-1,1,1,1,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,1,1,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,-1,0,0,-1,-1,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,-1,-1,-1,0,1,0,0,-1,0,0,0,1,1,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0],[0,0,0,-1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,-1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,-1,1,-1,-1,0],[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,-1,-1,1],[0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,1,0,-1,1,0,1,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,1,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,1,0,0,0,0,0],[0,0,1,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,-1,0,0,-1,1,0,0,-1,1,1,0,0,-1,0,0,-1,0,0,0,1,0,0,-1,1,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,-1,1,0,0,0,-1,-1,0,-1,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,1,0,1,0,-1,0,-1,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,-1,0,0,0,1,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,-1,1,0,-1,0,-1,0,0,0,-1,-1,-1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,1,0,0],[1,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,-1,1,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,-1,-1,0,0,-1,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,-1,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,1,-1,-1,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,1,1,0,0,0,-1,-1,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,-1,-1,-1,-1,0,0,1,0,0,0,1,0,-1,1,1,0,1,1,-1,1,0,-1,1,0,-1,0,-1,-1,0,0,-1,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,-1,1,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,0,-1,0,1,-1,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,0,1,-1,0,-1,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,1,-1,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,-1,0,0,1,0],[0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,-1,0,0,0,0,-1,-1,0,1,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,-1,0],[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,1,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,1,1,0,-1,1,0,1,1,0,1,0,0,0,1,0,0,0,-1,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,-1,0,1,-1,0,0,0,1,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,1,-1,0,0,-1,0,-1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,1,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,-1,1,-1,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,-1,1,0,-1,0,1,1,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,-1,1,0,1,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,-1,0,-1,0,0,-1,0,0,1,-1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,-1,0,-1,-1,0,0,0,0,1,0,1,-1,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,-1,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,0,0,0,0,0,-1,0,0,0],[0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,-1,0,-1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,-1,-1,0,0,1,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,1,1,-1],[1,0,0,1,-1,-1,-1,0,0,-1,1,0,1,0,0,0,-1,0,0,-1,1,-1,0,0,-1,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,1,0,0,1,0,0,0,-1,0,0,1,0,1,0,-1,0,0,0,-1,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,-1,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,1,1,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,1,1,1,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,1,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,-1,0,0,0,1,-1,-1,0,-1,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,-1,-1,0,1,-1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,1,0,-1,1,1,0,0,0],[0,0,0,0,0,1,1,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,-1,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,-1,-1,0,0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0],[-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,-1,0,-1,-1,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,-1,0,1,0,0,0,0,-1,0,0,0,-1,-1,0,-1,0,0,0,1,0,1,-1,0,0,0,0,0,-1,1,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,1,0,1,0,0,0,0],[0,0,0,0,-1,0,0,1,0,0,1,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,-1,1,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,1,0,1,0,1,1,0,0,1,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,-1,0,0,0,0,1,0,-1,0,1,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,-1,1,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,-1],[0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,-1,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,1,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0],[0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,-1,0,-1,0,0,1,0,0,0,0,-1,0,0,1,0,0,-1,0,0,-1,1,1,0,0,-1,-1,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,-1,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,-1,0,0,0,-1,0,0,1,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,1,0,0,1,1,0,0,-1,0,-1,0,1,0,-1,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,1,-1,0,0,-1,-1,0,0,0,1,0,0,1,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,-1,0,0,-1,1,0,1,0,0,0,-1,0,-1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0],[1,0,1,-1,0,-1,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,-1,0,0,0,1,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,-1,1,0,-1,1,-1,0,0,1,0,1,-1,1,0,0,-1,0,0,0,0,1,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,1,-1,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,-1,0,1,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,1],[0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1],[0,0,0,1,0,0,0,-1,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,-1,-1,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,-1,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,1,-1,-1,1,1,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,-1,1,0,0,0,-1,1,0,1,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,1,0,0,0,0,-1,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0],[0,0,0,0,0,0,-1,0,-1,0,0,-1,1,0,0,0,0,-1,-1,0,-1,0,1,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,-1,-1,0,0,0,0,1,1,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,1,-1,-1,0,0,0,0,0,0,0,1,0,0,1,0,1,1],[-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,-1,0,0,1,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,-1,-1,0,0,-1,-1,0,0,1,0],[0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,1,0,0,1,0,0,-1,1,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,-1,0,0,0,-1,0,1,0,-1,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0],[0,0,-1,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,-1,1,-1,0,0,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,1,-1,-1,-1,1,0,1,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,0,0,0,1,0,0,1,-1,-1,0,0,0,0,0,0,1,0,0,-1,0,-1,-1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,-1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,-1,-1,0,-1,0,0,0,-1,0,0,-1,1,-1,0,0,-1,0,0,0,1,1,1,0,0,1,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,1,0,0,1,0,1,0,-1,1,0,1,-1,-1,0,0,-1,1,-1,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,-1,0,0,0,0,1,1,0,0,1,0,1,-1,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,-1,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,-1,0,0,0,0,0,0,0,0],[0,0,1,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,-1,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,-1,-1,1,0,0,-1,0,1,0,0,0,1,0,0,-1,0,0,0,-1,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,-1,0,0,1,0,0,-1,0,1,0,0,-1,1,1,0,1],[1,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,1,0,0,0,1,-1,0,0,1,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,-1,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,1,0,0,1,-1,-1,0,0,-1,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,-1,1,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,1,0,-1,-1,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,-1,-1,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,-1,-1,0,-1,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,1,0,-1,-1,0,0],[1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,-1,-1,0,0,0,0,1,0,0,1,1,0,-1,0,0,0,1,0,0,0,1,0,-1,-1,0,1,0,0,-1,0,0,0,0,0,0,0,-1,1,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,-1,-1,0,1,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,1,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,1,0,0,0,-1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,1],[0,0,1,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,1,0,0,-1,-1,0,-1,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,-1,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,1,-1,0,-1,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,-1,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],[1,0,-1,1,0,1,0,0,-1,1,0,0,0,1,0,0,0,1,-1,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,1,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,1,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,1,1,0,-1,0,-1,1,1,0,-1,0,1,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,1,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,-1,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0],[0,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,-1,0,-1,0,1,1,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,1,0,1,0,0,1,0,0,-1,-1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,1,0,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,0,1,1,0,0,0,0,0,1,0,1,0,1,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,1,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,-1,0,0,1,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0],[-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],[1,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,-1,1,0,0,-1,1,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,-1,-1,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0],[1,1,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,-1,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,1,0,1,1,0,0,0,0,0,0,1,1,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,0,1,0,0,0,0,0,0,1,1,-1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,1,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0],[1,1,0,0,1,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,-1,0,1,0,0,0,1,0,1,0,1,0,0,1,0,-1,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,-1,-1,0,1,0,0,0,-1,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0],[0,1,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,-1,0,1,-1,0,0,1,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,-1,-1,0,0,0,0,-1,0,0,0,1,0,0,-1,-1,0,1,0,-1,1,0,-1,1,0,-1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,1,0,0,0,0,-1,0,0,1,0,1,0,1,1,1,0,0,0,-1,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,-1,1,-1,0,0,-1,0,0,0,1,0,0,1,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0],[-1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,-1,0,1,0,-1,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,1,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0],[1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,-1,-1,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,1,0,0,0,-1,0,0,-1,0,0,1,-1,0,0,0,-1,0,0,1,0,0,0,0,-1,0,1,0,-1,0,1,0,0,1,1,0,-1,0,1,0,-1,0,1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,-1,-1,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0],[0,0,0,-1,0,-1,1,0,-1,0,0,-1,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,-1,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,-1,0,0,-1,0,0,0,0,0,1,-1,0,1,0,-1,0,0,-1],[0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,-1,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,-1,0,0,1,0,-1,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,-1,-1,-1,-1,1,0,0,1,0,0,-1,1,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,1,0,-1,0,0,0,-1,0,0,1,1,-1,-1,0,0,0,-1,0,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,-1,0,0,1,0,0,1],[1,0,-1,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,1,-1,0,1,0,0,0,0,0,0,0,-1,1,0,1,1,-1,0,1,0,0,1,1,0,1,0,1,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,-1,0,1,0,-1,0,1,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,-1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0],[-1,0,0,-1,0,0,0,1,1,-1,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,1,1,1,0,0,0,0,1,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,1,1,0,0,1,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,1,0,1,-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,1,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,1,0],[0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,-1,0,-1,1,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,1,0,1,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,1,0,1,-1,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,1,0],[0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,-1,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,-1,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,-1,0,0,0,0,0,-1,-1,0,0,1,0,0,0,1,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,-1,1,0,0,0,-1,0,0,0,0,0],[1,1,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,-1,1,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,-1,0,-1,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,-1,0,-1,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,-1,0,1,-1,0,0,0,-1,0,0,0,1,1,0,1,0,-1,0,0,0,-1,0,1,-1,1,0,0,-1,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,-1,0,-1,1,0],[0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,-1,0,0,0,0,0,0,1,0,-1,-1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,-1,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,1,0,-1,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,1,0,0,0,0,1,0,-1,1,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,-1,0,0,0,1,0,0,0,0],[0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,1,0,0,-1,0,0,0,1,-1,0,0,-1,0,1,0,1,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0],[0,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,-1,1,0,0,0,1,0,0,0,-1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,-1,0,0,1,-1,0,-1,0,1,0,1,0,0,1,0,0,0,0,-1,0,0,1,1,1,1,-1,1,0,-1,0,0,1,0,-1,1,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,-1,1,0,0,-1,-1,1,-1,0,-1,-1,1,1,0,0,1,1,0,-1,0,1,-1,0,0,-1,-1,1,1,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,-1,0,0,0,0,1,0,1,-1,0,-1,0,0,1,0,0,1,-1,0,0,0,0,0,0,1,0,1,-1,0,0,0,-1,0,0,0,1,-1,1,0,1,0,0,1,0,1,0,1,-1,0,1,1,-1,-1,0,1,-1,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0],[0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,1,0,-1,1,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,1,1,-1,-1,0,0,-1,-1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,1,0,1,-1,0,0,-1,1,0,1,0,1,0,-1,0,0,0,-1,-1,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,1,0,1,-1,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0],[-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,-1],[-1,0,0,0,0,1,0,-1,-1,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,-1,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,-1,0,0,0,0,0,-1,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,0,1,0,0,1,-1,0,0,1,0,-1,0,1,-1,0,1,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,-1,0,-1,0,1,0,1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,-1,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0],[0,0,-1,0,0,0,1,0,0,-1,0,0,0,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,-1,1,1,-1,1,0,0,1,-1,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,-1,0,0,1,0,0,0,0,0,0,-1,0,0,1,-1,-1,1,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,-1,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,1,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,1,0,1,0,0,0,-1,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,-1,1,0,1,0,0,-1,-1,0,-1,1,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,1,1,0,1,1,0,0,1,0,0,0,0,0,-1,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,1,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,-1,0,0,0,1,0,1,0,0,1,1,0,0,1,0,-1,0,1,0,0,0],[-1,-1,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,-1,0,1,0,0,1,-1,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,-1,-1,0,1,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,1,0,-1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,1,1,1,0,1,0,1,0,0,0,0],[0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,1,1,0,0,0,1,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,1,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,0,0,-1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,1,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,1,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,-1,0,0,1,0,0,0,1,0,0,0,-1,1,1,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,1],[0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,-1,0,1,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,-1,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,-1,0,1,0,0,-1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,-1,0,0,1,0,0,0,0,0,0,-1,0,1,0,-1,0,-1,0,0,0],[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,-1,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,1,-1,0,0,0,1,0,1,1,0,0,-1,0,0,0,1,1,0,0,1,-1,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,1,-1,1,0,1,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,-1,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0],[-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,1,-1,-1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,1,0,-1,0,0,0,1,0,1,0,1,-1,0,0,1,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,-1,0,-1,-1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,1,-1,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,1,0],[0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,1,-1,0,1,-1,0,0,0,0,0,-1,1,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,1,0,-1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,-1,1,-1,0,0,1,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0],[-1,-1,0,0,0,0,0,-1,0,0,-1,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,1,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,-1,1,0,1,0,0,-1,0,1,1,0,0,0,0,0,0,-1,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,-1,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,-1,0,-1,-1,0,0,-1,1,0,0,0,0,0,-1,-1,-1,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,-1,0,0,1,0,1,0],[1,0,1,0,0,-1,1,1,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,1,0,0,0,1,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,1,0,-1,0,0,0,1,-1,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,-1,0,0,0,0,1,0,-1,0,0,1,0],[-1,-1,0,1,0,0,-1,0,-1,0,0,-1,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,-1,0,1,0,0,-1,0,0,0,1,0,0,-1,1,-1,1,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,-1]]
weights['h3']=[[0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,-1,0,-1,0,0,0,0,0],[0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,-1,0,0,0,1,0,1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,-1,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1],[0,1,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,1,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,1,-1,-1,0,0,0,0,-1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,1],[0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,-1,0,0,-1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1],[0,0,0,0,1,0,1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0],[0,-1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,-1,0,1,0,0,0,0,-1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0],[1,0,0,-1,0,0,0,0,0,1,-1,0,-1,0,0,-1,0,1,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,-1,1,-1,0,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,1,0,0,0,1,1,0,-1,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0],[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,-1,0,1,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,1,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],[1,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,-1,1,0,0,1,0,-1,0,0,0,0,0,0,0,1,1,-1,0,-1,1,0,-1,1,0,0,0,0,1,0,1,1,0,0,0,0],[1,1,0,0,0,0,-1,0,1,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,1,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,-1,-1,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,-1,0,-1,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,-1,0,0,1,0,0],[0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,-1,0,-1,-1,1,0,-1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,1,0,-1,0,0,-1,0,0,0,-1,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,-1,1,1,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,1,-1,-1,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,1,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[1,0,-1,-1,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,-1,0,-1,1,1,0,0,0,-1,0,0,0,-1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,1,-1,0,-1,0,0,0,0,0,-1,0,1,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,1,-1,-1,-1,0,0,-1,-1,0,0,0,0,0,-1,-1,0,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,-1,-1,0,0,0,0,-1,0,0],[0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,1,-1,0,-1,1,-1,0,0,0,0,0,-1,0,-1,0,1,0,0,0,1,0,-1,0,0,0,-1,0,0,0,0,0,1,-1,0,0,-1,1,0,-1,1,0,0,1,0,0,0,-1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,-1,-1,1,0,1,0,0,0,0,-1,1,0,0,0,-1,0,-1,0,1,-1,0,0,0,0,-1,-1,0,0,1,1,0,0,0,0,-1,1,1,0,1,1,-1,0,1,0,1,0,1,0],[-1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,1,-1,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,-1,1,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1],[1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,1,-1,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0],[0,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1],[0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,1,0,0,-1,0,1,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,-1,-1,1,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,1,-1,0,0,0,0,0,-1,0,0,1,0,0,0],[-1,0,-1,0,0,0,0,0,-1,0,1,1,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,-1,0,0,1,1,0,0,0,0,-1,0,0,1,0,1,-1,0,-1,-1,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,-1,1,0,-1,0,0,0,0,-1,-1,0,0,-1,-1,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,-1,1,0,0,-1,0,0,1,0,0,0,-1,-1,0],[0,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,-1,0,0,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0],[0,0,-1,-1,0,0,0,0,0,1,-1,0,0,0,-1,-1,0,0,0,1,0,1,-1,0,1,0,0,-1,1,0,0,0,1,0,0,0,0,0,1,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,1,-1,1,0,0,1,0,1,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0],[1,0,0,-1,1,1,0,1,1,0,0,-1,-1,0,0,0,1,1,1,-1,0,-1,0,0,0,0,1,-1,0,1,-1,0,1,0,0,-1,0,0,1,0,0,0,0,-1,-1,0,0,0,-1,0,-1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,-1,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,-1,0,-1,-1,-1,1,-1,0,0,0,-1,1,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,1,0,-1],[0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,1,1,0,0,0,0,1,0,1,1,0,1,0,-1,0,1,0,-1,0,0,0,-1,1,0,1,0,0,0,-1,-1,0,-1,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,-1,0,-1,0,0,0,0,0,0,1,1,-1,0,1,-1,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,0,0,0,1,0,1,0,-1,0,0,-1,0,1,0,0,0,1,0,1,0,0,0],[0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,-1,0,0,0,0,0,-1,1,1,0,0,0,0,0,1,-1,1,0,-1,-1,0,1,0,0,1,1,-1,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,-1,1,0,0,1,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,1,-1,0],[0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],[1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,-1,-1,0,1,1,0,1,0,0,0,1,-1,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,-1,-1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,-1,-1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,-1,0,0,0],[1,0,-1,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,-1,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,-1,0,0],[0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,1,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,-1,-1,0,0,0,-1,1,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,-1],[0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,-1,-1,0,1,0,0,1,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0],[1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-1,0,-1,0,0,1,-1,0,0,1,-1,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,-1,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,-1,0,1,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0],[0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,1,-1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,-1,0,1,0,1,1,1,0,-1,-1,0,0,0,-1,-1,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1],[0,0,0,-1,0,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,-1,1,1,0,1,0,0,0,0,0,-1,1,0,1,0,-1,1,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,-1,0,0,1,-1,0,1,0,0,-1,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,-1,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1],[0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,-1,0,0,-1,0,-1,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1],[0,0,0,1,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,-1,-1,0,1,0,0,0],[0,0,0,1,1,0,0,-1,0,0,1,1,-1,-1,0,0,0,1,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,-1,-1,1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,1,0,-1,0,-1,1,-1,0,0,0,1,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,1,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,-1,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,1,1,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0],[0,1,0,0,0,1,0,0,1,0,1,0,0,-1,0,1,0,-1,0,0,0,0,0,1,0,0,1,0,0,-1,1,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0],[0,0,1,1,0,0,0,-1,0,0,1,0,0,1,0,1,0,-1,0,0,0,0,1,1,1,0,0,-1,-1,0,0,-1,0,1,0,0,0,-1,0,-1,1,1,0,1,0,0,0,0,0,-1,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,-1,-1,0,0,0,0,-1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,-1,1,0,0,1,1,-1,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,1,0,1],[0,0,-1,0,0,1,0,1,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0],[0,0,1,0,-1,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,1,-1,-1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0],[-1,0,0,0,0,0,1,-1,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,-1,1,0,1,0,0,0,0,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,-1,0,1,0,-1,1,1,0,0,0,1,1,-1,0,0,0,0,0,1,-1,1,-1,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0],[0,-1,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,-1,-1,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,1,-1,0,1,0,0,0,1,0,0,0,1,0,0,-1,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,-1,0,0,1,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,-1,0,0],[1,0,0,0,-1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0],[0,0,-1,0,-1,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1],[0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,-1,0,1,0,1,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,-1,1,0,0,1,1,-1,0,0,0,0,1,1,-1,0,0,0,1,1,0,0,0],[1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,-1,0,1,0,0,0,0,0,-1,0,0,1,-1,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0],[-1,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,-1,0,-1,0,-1,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,1,0,0,0,-1,0,-1,0,0,-1,0,0,0,0],[0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,0,-1,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,-1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,-1,1,0,0,-1,-1,0,0,1,-1,1,0,0,0,0,0,-1,0,1,-1,0,0,0,0,0,0,0,1,0,0,0,1,1,-1,0,0,-1,-1,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,1,-1,0,-1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,1,0,1,1],[0,0,0,0,0,-1,-1,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,1,1,0,0,-1,0,0,0,0,1,-1,-1,0,0,0,0,1,0,1,0,-1,1,-1,1,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,1,0,-1,0,0,0,-1,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1],[0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,-1,0,0,0,0,-1,0,0,0,-1,0,1,1,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,1,1,-1,0,0,1,1,1,0,0,0,-1,1,1,0,0,-1,0,1,-1,-1,0,1,0,-1,1,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,1,0,0,1,0,0,0,-1,1,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,-1,0,0,-1,0,1,0,-1,-1,0,0],[0,0,1,0,0,0,0,0,1,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,-1,1,-1,0,-1,0,0,-1,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,0,-1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,1,0,-1,0,0,0,0,-1,0,-1,0,0,0,-1,1,0,1,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,1,1,0,-1,0,1,-1,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,-1,-1,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,-1,1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,1,-1,1,0,-1,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1],[0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0],[0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,-1,-1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1],[0,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,-1],[0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,1,0,1,0,1,0,0,0,0,1,-1,0,0,0,1,-1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,-1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,-1,0,1,-1,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,-1,-1,0,0,0,0,0,-1,1,-1,1,1,-1,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],[1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,1,1,1,1,1,0,-1,0,0,1,0,0,-1,0,0,1,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0],[0,0,-1,0,0,0,-1,1,0,1,0,0,0,0,-1,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,-1,0,-1,0,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,1,0],[-1,-1,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,-1,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,1,0,0,1,0,0,0,-1,0,0,1,-1,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0],[0,-1,1,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,-1,0,0,0,1,0,-1,0,0,-1,1,0,-1,0,1,0,0,0,0,0,0,1,0,0,1,-1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1],[0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,1,0,0,-1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0,-1,-1,-1,0,0,0,0,1,-1,0,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,0,-1,1,-1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0],[0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0],[1,-1,-1,0,-1,-1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,1,0,-1,0,1,0,0,0,1,-1,-1,0,0,0,-1,0,1,1,0,0,1,0,0,1,1,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,-1,1,-1,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0],[1,0,0,0,0,0,-1,1,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,-1,0,0,-1,0,-1,0,0,-1,0,0,-1,0,0,1,0,0,0,-1,1,0,-1,1,0,-1,-1,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,-1],[0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,-1,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,1,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,-1,0,0,-1,0,0,0,0,1,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,-1,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,-1,0,0],[1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,1,-1,0,-1,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0],[0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0],[-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,0,-1,0,0,0,-1,1,1,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0],[0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,-1],[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,-1,-1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0],[0,0,0,0,0,0,1,1,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,-1,1,0,0,0,0,1,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0],[0,0,0,0,-1,-1,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,1,1,1,0,0,0,1,0,0,1,-1,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],[0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,-1,0,-1,0,1,0,0,0,-1,0,0,0,0,-1,1,0,0,-1,0],[0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,-1,-1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,-1,0,-1,0,0,0,0,1,-1,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,1,-1,0,0,0,-1,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,-1,1,1,0,0,1,1,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,1,0,0,0,0,-1],[-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,1,-1,0,0,0,0,-1,0,1,0,1,0,0,0],[0,0,0,0,0,1,0,1,-1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,-1,0,0,-1,0,0,0,0,-1,1,0,0,-1,0,0,0,0,1,-1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,1,1,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,-1,1,0,0,1,0,0,-1,1,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,1,0,-1,-1,0,0,0,0,0,0,1,1,1,0,-1,1,0,-1,-1,0,0,0,-1,0,0,-1,0,0,-1],[0,1,0,-1,0,0,1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,1,-1,0,0,1,1,0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,-1,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,-1,0,-1,-1,0,0,0,0,-1,-1,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0],[-1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,-1,0,0,0,-1,-1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0],[0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,1,1,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,1,0,0,0],[1,0,1,0,0,0,-1,1,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,-1,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,1,0,0,0,1,0,0,0,-1,0,1,-1,0,1,0,0,0,0,0,0,0,0],[0,0,0,0,1,-1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,1,1,0,1],[0,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,1,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,1,-1,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,1,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0],[0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,-1,1,-1,0,0,0,1,0,0,0,-1,0,-1,0,-1,0,0,1,0,0,0,0,-1,-1,0,0,0,0,-1,0,0,-1,0,1,0,0,1,0,0,0,0,1,0,1,-1,0,0,1,0,0,1,1,0,0,0,1,-1,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,1,1,-1,0,0,0,0,0,-1,-1,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,-1,-1,0,1,0,0,-1,-1,0,0,-1,0,0,0,1,0,0,1,0],[0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,1,-1,-1,0,0,1,0,-1,1,0,0,0,0,0,-1,1,0,0,0,0,1,0,0,0,-1,0,0,-1,-1,0,1,1,-1,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,-1,0,0,-1,0,0,-1,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,1,0,-1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,-1,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,1,-1,0,0,1,0,0,-1,0,0,-1,-1,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,-1,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,1,0,-1,-1,0,0,0,1,0,1,-1,0,-1,0,-1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0],[0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,-1,-1,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,-1,1,0,0,-1,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0],[0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1,0,1,0,1,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,-1,-1,0,-1,0,0,0,0,-1,-1,0,0,0,-1,0,-1,0,-1,0,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],[0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,0,-1,0,0,0,0,-1,-1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,-1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1],[-1,-1,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,-1,-1,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,1,-1,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,-1,0,0,1,0,0,0,0,0,1],[0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,0,-1,-1,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,-1,0,-1,1,0,0,0,0,1,0],[0,0,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0],[-1,0,0,0,0,1,0,0,0,0,0,-1,0,-1,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,-1,0,-1,1,-1,1,0,0,1,-1,0,-1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,1,0],[0,0,0,1,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,-1,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,1,0,0,-1,0,0,0,-1,0,0,0,0,-1,0],[0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,-1,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,-1,0,1,0,0,0,1,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,1,0,-1,0,0,1,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,-1,1,0,-1,1,1,0,0,0,0,-1,-1,0,0,0,-1,0,-1,-1,-1,1,0,0,0,-1,0,0,-1,0,0,0,0],[-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,1,-1,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1],[0,-1,0,1,0,-1,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-1,-1,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,-1,-1,-1,0,1,-1,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0],[0,1,0,0,0,0,1,-1,0,0,1,0,0,0,-1,1,0,0,0,0,1,0,0,0,-1,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,1,-1,1,0,0,0,0,1,0,0,-1,0,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,1,-1,0,0,-1,0,0,0,-1,0,0,1,-1,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,-1,-1,0,0,0,-1,0,1,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,1],[0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,-1,0],[0,0,1,0,0,-1,0,0,0,-1,0,0,-1,-1,1,-1,1,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,1,0,1,0,0,1,1,0,0,0,-1,1,0,1,-1,-1,0,0,0,0,1,0,1,0],[0,1,0,0,0,0,1,0,0,0,0,0,-1,0,-1,1,0,0,0,0,1,0,1,1,0,0,0,-1,1,-1,-1,0,0,0,0,0,0,0,-1,0,-1,1,-1,0,0,0,0,0,0,0,0,-1,1,0,-1,1,-1,0,0,0,0,-1,0,-1,-1,0,1,0,0,0,0,0,1,0,0,0,-1,0,-1,1,1,1,0,0,0,0,0,0,1,1,0,0,-1,-1,-1,0,-1,1,0,1,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,1,0,-1,0,1,0],[1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,0],[0,0,1,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,1,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,-1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,-1,0,1,-1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,-1],[0,0,-1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,-1,0,0,0,-1,0,-1,0,0,-1,0,1,0,0,1,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0],[-1,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0],[0,-1,0,0,-1,0,0,-1,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,-1,1,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,-1,1,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,-1,0,1,0,0,1,1,0,-1,0,0,0,0,-1,0,0,-1,0,0,1,1,-1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,-1],[1,0,1,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,0,0,0],[0,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,1,0,1,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,1,-1,0,0,0,-1,-1,1,-1,0,1,0,0,0,-1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,1,0,0,0,0,1,0,-1,1,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,1,0,0,-1,0,1,-1,0,0,0,0,-1,0,-1,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,1,-1],[0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,-1,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,-1,0,1,0,0,0,1,0,0,-1,-1,1,0,0,0,0,0,0],[0,1,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,1,-1,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,1,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,-1,1,0,0,0,0,0,1,1,0,0,0,0,1,-1,0,0,0,0,0,0,0],[0,0,0,0,0,-1,-1,0,-1,1,-1,0,-1,1,-1,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,0,0,0,0,-1,-1,-1,0,-1,0,0,1,-1,0,0,0,0,0,1,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,-1,0,1,0,-1,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,-1,1,0,-1,0,0,0,1,0,0,-1,0,0,0,-1,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0],[0,0,0,1,0,0,0,0,1,1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,1,0,1,-1,0,0,-1,0,1,-1,0,0,-1,-1,0,0,0,0,0,1,0,-1,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[0,1,0,0,0,-1,-1,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,1,0,-1,0,0,0,-1,1,0,1,0,0,-1,-1,0,1,1,0,0,1,1,-1,-1,0,1,0,1,0,1,-1,0,1,-1,0,0,0,-1,0,0,1,0,0,0,0,-1,0,1,-1,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,-1,0,1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,-1],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,-1,-1,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,1,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,-1,1,-1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,-1,0,1,0,-1,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,1,1],[0,0,0,1,0,0,0,0,0,0,1,0,-1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,-1,1,1,0,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0],[-1,0,1,0,-1,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,-1,0,0,-1,0,1,-1,0,0,0,1,0,0,-1,0,0,0,0,-1,0,1,0,0,1,1,1,0,1,0,0,0,1,-1,0,0,0,-1,1,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,1,0,0,-1,0,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,-1,-1,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,1,-1,0,0,0],[0,0,0,-1,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,-1,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0],[0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,-1,1,0,1,0,-1,-1,0,0,0,0,0,0,0,0,1,1,-1,1,-1,0,1,1,0,1,-1,0,0,0,0,0,-1,0,-1,0,1,0,-1,0,0,0,0,0,-1,0,0,-1,-1,0,1,0,-1,0,1,0,0,-1,0,0,0,0,-1,1,0,0,1,0,1,0,-1,0,0,0,1,0,0,1,0,0,0],[-1,1,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,0,1,0,0,0,-1,1,0,-1,0,0,0,0,0,-1,0,-1,0,0,-1,1,0,0,1,1,1,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,1,1,-1,0,1,0,0,0,0,0,1,0,1,0,0,-1,0,1,0,0,1,-1,1,0,-1,-1,-1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,1,1,1,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,1,0,-1,-1],[1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,1,1,0,0,0,0,-1,1,0,0,0,0,0,-1,1,0,0,0,1,-1,0,-1,1,0,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,-1,1,0,0,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,-1,0,1,-1,0,0],[0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,-1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,-1,1,-1,0,0,0,0,-1,-1,-1,0,0,0,1,0,0,1,-1,-1,0,1,0,0,1,0,0,0,-1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,-1,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,-1,0,0,-1,1,1,1,0,0,-1,1,0,1,0,0,0,0,0,0,0],[0,1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0],[1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1],[0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,1,-1,0,-1,0,0,1,-1,0,0,0,0,-1,-1,0,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,1,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,-1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,-1,0,0,1,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,1],[0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0],[0,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,-1,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,-1,0,1,0,0,1,0,0,-1,1,1,1,0,0,-1,0,1,0,-1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,-1,0,1,-1,0,0,-1,-1,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,-1,0],[0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,1,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1],[0,-1,0,0,0,-1,-1,0,0,0,1,0,0,0,0,1,-1,1,-1,1,0,-1,1,0,0,0,-1,0,1,1,0,1,-1,0,-1,0,-1,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,-1,-1,1,0,0,-1,-1,0,0,0,1,0,1,0,0,0,0,0,-1,-1,1,0,0,0,0,-1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1],[0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1],[0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,1,-1,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,1,0,0,1,-1,0,0,0,0,0,-1,0,-1,0],[0,0,1,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0],[1,0,0,0,-1,0,0,-1,0,0,0,1,0,1,0,0,1,0,0,1,-1,0,0,0,0,0,-1,-1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,1,0,-1,0,-1,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,-1,0,1,1,0,-1,0,-1,0,0,0,0,0,-1,0,0],[1,0,0,0,0,0,-1,0,0,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,0,0],[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0],[1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,-1],[0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,1,1,-1,1,0,0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0],[0,0,-1,-1,0,0,0,1,-1,0,0,-1,0,0,1,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,1,-1,1,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,1,0,0,-1,0,1,0,0,-1,-1,0,0,0,1,0,1,0,1,-1,0,0,0,0,1,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,-1,0,0,-1,1,1,0,1,0,0,0,-1,-1],[0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,1,-1,0,1,-1,0,0,-1,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,1,-1,0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,-1,1,1,0,0,-1,0,0,0,0,-1,0],[0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,-1,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,-1,0,0,0,0,1,1,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,-1,0,-1,0,1,0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,1,-1,0,0,-1],[0,0,0,1,-1,0,0,-1,0,0,1,0,0,0,0,-1,0,0,1,-1,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,0,-1,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,1,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],[-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,-1,0,1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,-1,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,-1,0,1,0,0,0,0],[0,0,0,0,1,0,1,-1,0,0,0,0,0,0,0,0,1,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,0,0,0],[1,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,-1,0,1,0,0,0,1,0,0,-1,0,0,-1,1,0,0,1,-1,0,0,0,-1,1,-1,0,0,0,0,0,0,-1,1,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,1],[-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,1,0,-1,1,0,1,-1,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,-1,0,0,-1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,-1,0,0],[0,0,1,0,0,0,0,1,0,1,1,-1,0,1,0,-1,0,0,-1,0,1,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,1,0,0,0],[0,1,0,0,0,0,-1,1,-1,0,1,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,0,0,1,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,1,-1,0,0,0,1,0,0,-1,0,-1,-1],[0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,1,-1,0,0,1,-1,0,-1,0,0,-1,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,-1,0,-1,0,0,0,0,0,0,-1,-1,0,0,0,-1,0,0,1,-1,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,1,-1,0,0,-1,0,1,0,0,0],[0,0,1,0,0,0,-1,0,0,0,1,-1,0,1,0,1,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,-1,-1,-1,0,0,1,0,0,1,0,0,-1,-1,0,-1,0,0,0,0,0,0,1,1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,-1,1,1,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,-1,-1,0,0,1,0],[0,-1,0,0,0,0,1,0,0,0,0,1,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,-1],[0,0,0,0,0,0,-1,1,0,0,0,-1,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,-1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,1,0,0,-1,0,-1,0,-1,-1,0,0,1,-1,1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,-1,0,0,0,1,1,1,1,0,-1,0,0,0,1,0,0,0,0,1],[0,0,0,0,1,0,0,0,0,0,0,0,-1,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0],[0,0,1,1,0,-1,0,0,0,1,0,0,0,1,-1,0,1,0,0,0,0,-1,0,0,0,0,-1,-1,1,0,0,-1,0,-1,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,-1,0,1,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,1,0,1,1,0,0,0,0,0,1,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,-1,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,1,0,0,0,1,0,0,1,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,0],[0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,1,0,1,1,0,-1,0,1,0,0,0,-1,0,0,0,0,1,0,0,1,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,-1,0,-1,0,-1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,0,0,1,1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,-1,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,-1,0,0,-1,1,-1,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,0,0,1,1,-1,-1,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,1,1,0],[0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,-1,0,0,-1,1,0,0,-1,0,0,-1,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-1],[0,-1,-1,0,0,-1,0,1,0,0,0,0,0,1,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,1,-1,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,1,0,0,1,0,0,0,0,0],[1,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,1,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,1,-1,0],[-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,0,1,1,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0,-1,0,1,1,0,0,0,1,-1,0,0,0,-1,1,-1,0,0,0],[0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,1,0,-1,1,-1,0,0,0,1,-1,0,0,0,0,1,0,0,0,1,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,-1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[-1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,-1,0,0,1,1,-1,0,0,0,0,-1,0,1,0,0,-1,-1,0,-1,-1,-1,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,-1,1,-1,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0,-1,0,-1,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,1,0,0,-1,1,-1,0,0,-1,0,0,0,0,-1,0],[0,-1,0,0,0,1,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0],[0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,0,0,0,-1,-1,0,1,-1,0,1,-1,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,-1,-1,0,1,0,0,1,0,0,0,0,-1,-1,0,0,1,0,1,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,1,0,0,1,0,1,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,-1,-1,-1,0,0,-1,0,0,0,0,-1,-1,0,0,1,1,0,-1,1,0,0,1,-1,0,0,-1,0,-1,0,-1,0,-1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0],[-1,0,0,1,0,1,0,0,0,-1,1,0,-1,0,1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,-1,0,0,0,0,-1,0,-1,0,-1,0,0,0,-1,0,0,0,1,-1,0,1,0,0,1,0,0,0,-1,0,-1,0,-1,0,0,0,0,1,1,1,-1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,-1,0,0,-1],[0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,1,1,0,0,0,1,0,1,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,-1,1,0,0,-1,-1,0,0,0,0,0,1,0,-1,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1,1,0,0,0,-1,0,1,0,-1,-1,0,0,0,0,0,-1,-1,0,-1,0,0,0],[0,0,0,0,1,0,0,-1,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,-1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1],[0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,-1,0,-1,0,0,0,-1,0,-1,-1,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,-1,1,0,1,-1,1,0,0,0,1,0,0,0,-1,0,0,-1,0,1,0,0,0,1,-1,0,0,1,1,0],[0,-1,0,0,0,0,0,1,1,0,0,1,1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,1,1,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,-1,0,1,0,-1,0,0,0,1,0,0,0,-1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,-1,-1,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,-1,1,0,1,0,-1,1,-1,0,-1,-1,0,-1,0,1,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,1],[0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,1,1,-1,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,1,0,-1,-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,-1,0,0,-1,-1,0,-1,0,0,0,0,0,0,0,1,0,0],[0,1,-1,1,-1,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,-1,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,0,0,-1,1,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,-1,1,0,0,0,-1,0,1,-1,1,0,0,0,-1,0,0,0,0,1,0,-1,1,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,1,0,-1,1,0,0,0,1,1,-1,1,0,0,0,1,1,0,1,0,0,-1,0,0,-1,0,0,1,0,-1,1,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,1,-1,0,1,-1,1,1,1,0,0,0,0,0,1,0,0,0,0],[0,0,0,0,-1,0,0,0,0,0,1,-1,-1,0,0,1,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,-1,0,1,0,1,0,0,-1,1,0,0,1,-1,0,-1,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0],[0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,-1,1,0,0,-1,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,-1,0,1,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,1,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,-1,-1,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,-1,-1,1,0,0,0,-1,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,-1,0,1,1,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,1,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,1,0,-1,0,0,0,0],[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,1,0,1,0,0,-1,0,0,0,0,-1,0,1,0,0,-1,0,0,-1,0,-1,0,0,1,0,0,0,0,-1,0,1,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,1,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,0,0,0,1,-1,0,1,-1,0,0,0,1,0,0,0],[1,0,0,0,0,-1,-1,-1,0,0,-1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,-1,0,0,-1,0,0,1,1,1,0,0,1,0,0,0,0,-1,0,0,-1,1,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,1,1,0,1,-1,0,0,1,0,1,0,0,0,0,0,0,0,-1],[-1,1,0,0,0,0,0,0,-1,0,0,0,-1,1,0,1,1,0,0,-1,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,-1,0,0,1,0,0,1,-1,0,0,-1,0,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,1,-1,1,1,0,0,1,-1,0,0,0,0,0,-1,1,0,0,0,0,-1,1,1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,1,0,0,1,0,1,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,-1,-1,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,1],[0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,-1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,1,0,0,1,-1,1,0,-1,-1,-1,1,0,0,0,0,1,0,-1,1,0,-1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,1,-1,0,0,0,0,1,-1,1,-1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,-1,1,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,1,-1,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1]]
weights['h4']=[[0,0,1,0,-1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0],[1,0,0,0,0,0,0,1,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,1,-1,1,1,0,0,0,0,0,-1,1,0,0,-1,0,0,1,-1,1,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0],[1,-1,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,1,0,0,1,0,0,0,0,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,-1,0,0,0,0],[0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,1,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,-1,0,1,0,0,0],[-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,-1,-1,0,0,-1,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,-1,-1,0,0,0,0,0],[0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,1,-1,1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1],[0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,0,0,0],[0,0,-1,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,-1],[0,0,0,0,0,-1,1,0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,-1,0,1,0,0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1],[0,0,1,0,0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,-1,1,0,0,0,0,1,1,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0],[0,0,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,1,0,1,0,0,0,0,0,-1,0,0,1,-1,0,-1,1,0,0,0,0,0,-1,0,0,1,0,0,-1,-1,0,0,0,0,0,-1,0,1,0,0,0,-1,0,0,0,1,0,0,0],[0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0],[0,0,-1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0],[0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,1,0,1,1,0,-1,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,-1],[0,0,1,1,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,1,-1,-1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0],[-1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,1,-1,0,0,0,0,0,-1,0,0],[1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,0,0,0,-1,1,0,0,0,0,0,-1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,-1,0,0,-1,0,0,0,0,-1,0,-1],[0,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,1,0,1,-1,-1,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,0,-1,0,1,0,0,-1,0,0,0],[1,0,1,0,1,-1,-1,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,-1,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0,-1,0,-1,0,0,1,0,0,-1,0,0,0,0],[1,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,1,0,0,0,-1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,1],[1,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,1,-1,1,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1,0,0],[0,0,0,1,-1,0,0,-1,-1,-1,0,1,0,0,0,1,-1,0,1,0,0,-1,0,0,0,0,-1,0,-1,0,0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1],[0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1],[1,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,0,0,0],[-1,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,-1,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,0,1],[0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,-1,-1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,-1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0],[0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,-1,1,0,1,0,0,0,0,0,0,0,1,1,0,0,-1],[0,-1,0,0,0,0,0,1,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0],[0,0,0,1,0,0,0,1,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1],[0,1,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,-1,1,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,1,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0],[0,0,0,0,0,0,-1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,-1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,-1,0,1,0],[0,0,0,0,1,0,0,0,0,-1,1,0,1,0,1,0,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,-1,0,-1,0,0,0,0,0,0,0,0,1,-1,0,-1,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,-1,0,1,0,0,1,0,-1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0],[0,-1,0,-1,0,1,0,0,1,1,0,0,0,0,1,0,0,-1,-1,0,1,-1,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0],[0,0,0,0,1,1,0,-1,-1,0,0,0,0,0,1,0,-1,0,0,0,0,1,-1,0,1,1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0],[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,1,-1,0,-1,0,-1,1,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,-1,1,0,0,0,0,0],[0,1,0,-1,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1],[0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,1,0,1,0,0,1,0,0,-1,-1,0,0,1,0,1,0,0,0,0,0,1,1,0,-1,1,0,0,0,0,0,1,0,1,0,1,0,0,-1,0],[0,1,0,1,0,0,-1,0,0,0,0,1,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,1,-1,1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0],[0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0],[0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,1,-1,0,0,0],[0,0,0,0,-1,-1,0,0,0,0,-1,0,-1,1,1,0,0,-1,1,0,1,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,0,0,-1],[-1,0,-1,0,0,-1,1,0,-1,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,-1,0,-1,0,-1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0],[0,-1,0,0,0,0,1,-1,1,0,-1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,-1,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,0,-1,0,0,0,0,0,1,0,-1,0,0,0,1,-1,0,0,0,0,0,-1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,-1,0,-1,1,0,1,-1,1,0,0,0,0,0,1,0,-1,0,0,-1,-1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,1,0,1,0],[-1,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,-1,-1,0,-1,0,0,1,0,1,0],[-1,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,-1,1,-1,-1,0,0,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,1,-1,1],[-1,0,0,0,-1,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,1,0],[1,-1,0,-1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,1,1,0,0,0,1,-1,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0],[0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0],[0,0,0,1,1,0,0,0,1,-1,0,0,1,0,-1,0,0,1,0,0,-1,0,0,0,-1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,-1,-1,0,0,1,0,0,0,0,0,1,0,0,-1,-1,0,-1,0,0],[-1,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0],[0,0,0,0,1,0,0,-1,0,-1,0,0,1,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,1,0,-1,0,1,-1,0,0,0,0,0,0,0,0,1,0,1,0,-1,0,0,0,0,0],[0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,-1,0,0,0,1,-1,0,1,0,0,-1,0,0,0],[0,1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-1,-1,0,0,0,0,0,0,-1,0,-1,0,0,1,0,0,0,0,0,1,0],[0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],[-1,0,0,0,-1,0,0,0,-1,0,0,0,0,1,0,0,-1,0,-1,0,0,0,0,-1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,1,0,-1,0,0,1,0,0],[0,1,-1,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0],[1,0,-1,0,0,-1,0,0,0,0,1,0,-1,0,1,0,0,0,-1,0,0,0,1,0,1,-1,0,0,0,1,0,0,1,0,1,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1],[0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,-1,0,0,0,-1,1,0,1,-1,0,-1,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,1,-1,0,0,0,0,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0],[0,-1,0,0,0,0,0,0,0,0,0,1,1,0,-1,-1,0,1,-1,0,0,-1,0,1,0,0,0,0,-1,-1,1,1,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,-1,0,0,0,0,0,-1,1,1,-1,0,1,0],[0,1,0,0,0,0,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,1,1,0,1,-1,-1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,-1,1,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,1,1,0,1,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,0],[1,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,-1,0],[0,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,-1,0,0,-1,1,-1,0,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,0,-1,0,-1,0,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,-1,0,0,1,0,0,0,1,0],[0,1,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1,1],[0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,-1,-1,-1,0,1,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,-1,-1,1,0,0,0,1,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,-1,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],[0,-1,0,-1,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,-1,-1,1,1,-1,0,0,-1,0,0,-1,0,0,1,-1,0,-1,0,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,0,0,-1,0,0,-1],[0,-1,0,0,0,0,0,0,1,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,0,1,0,0,0,0,-1,0,0,1,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,1,0,0,-1,0,1,0,0,0,0,0,0],[0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,-1,0,1,1,-1,1,0,0,-1,0,1,0,0,-1,0,0],[1,1,1,-1,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,-1,0,-1,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,-1,-1,1,0,0,-1,0,0,1],[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0,1,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,-1,-1,0,0,-1,0,0,0,1,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,-1,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,-1,0,1,0,1,0,0,-1,0,0,0,0],[0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,1,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0],[0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,-1,0,0,0,1,0,-1,0,-1,0,1,0,0,0,-1,0,0,0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,-1,-1,0,0,0],[0,-1,0,1,1,-1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,1,-1,0,0,0,-1,0,0,0,0,1,1,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,-1],[0,-1,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,1,0,0,0,1,-1,0,0,0,0,0,0,-1,-1,1,0,1,0,0,0,0,-1,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,-1,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,-1,1,0,1,0,0,0,0,-1,0,0,0,1,0,0,-1,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0],[1,0,1,0,0,0,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,-1,0,0,1,0,0,0,1,-1,1,0,0,1,0,0,1,0,-1,0,0,0,0,1,1,-1,0,0,0,0,0,0,0,0,1,1,1,0],[-1,0,0,0,1,1,0,0,0,0,1,-1,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,-1,0,0,0,1,0,0,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,-1,0,1,0,0,0],[-1,0,0,0,0,0,0,0,0,-1,0,-1,0,0,-1,-1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,-1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0],[0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,-1,0,0,-1,0,0,0,0,1,0,1,-1,0,0,0,1,-1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,1,0,0,0,0,0,1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,-1,0],[0,0,0,1,0,0,-1,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,-1,1,0,-1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,-1,0,0,0,-1,-1,0,1],[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0,0],[-1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,-1,0,-1,1,0,-1,-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,1,0,-1,0,0,0,1,1,1,0,0,0],[0,1,1,1,0,0,0,-1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,-1,0,1,0,0],[0,-1,0,0,0,1,1,0,0,1,0,-1,-1,0,1,0,0,-1,1,0,-1,0,0,1,1,0,1,1,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,-1,0,1,0,-1,0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0],[0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,-1,0,-1],[0,1,0,0,0,-1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,1,-1,0,-1,0,0,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,-1,0,0,1,1,-1,1,0,0,1,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,1,0,0,0,0,-1,1,-1,0,1,0,-1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,0,1,1,0,0,0,0,1,-1,0,0,0,0,0,0],[-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,-1,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,1,0,-1,-1,0,0,0,0,0,1,0,-1,0,0,0],[-1,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,1,-1,0,0,0,0,0,1,0,1,0,0,-1,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,1,0,0,0,0],[0,1,1,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,-1,0,0,0,-1,0,-1,0,1,0,0,0,1,0,-1,-1,0,-1,0,0,-1,0,1,0,1,0,0,0,-1,0,-1,0,0,0,1,0,0,-1,-1,0,1,0,0],[-1,0,-1,0,-1,0,0,0,0,-1,0,-1,0,1,0,0,0,0,0,-1,0,0,0,1,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,-1,0,0,0,0,0,1],[0,-1,1,1,0,-1,0,0,-1,0,0,0,1,-1,0,0,0,1,-1,0,0,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,-1,0,0,0,0,-1,1,1,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0]]
weights['h5']=[[1,0,1,0,0,0,0,0,1,0,0,-1,0,-1,-1,0,0,0,0,0,0,0,1,0,0,-1,1,-1,1,-1,0,0],[-1,-1,1,0,0,0,0,0,0,1,0,-1,0,0,1,0,1,0,-1,1,0,0,0,0,0,0,0,0,-1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,-1,-1,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,1,0,1,0],[0,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0],[0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,1,1,0,0,0,0,0,0,0,0,-1],[1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,-1,0,0,1,0,-1,0],[0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,-1,0,-1,1,1],[-1,0,0,-1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,1,0],[0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,-1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1],[0,0,0,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1],[-1,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,1],[0,-1,0,0,1,0,0,0,0,-1,-1,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,1,0,0,0,1,0,-1,-1,0,0,0,0,0,0,0,0,0,1],[0,0,0,0,0,0,1,0,0,0,0,1,0,1,-1,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,-1],[0,1,1,0,0,0,0,0,1,-1,0,0,0,0,0,-1,0,1,0,0,1,0,1,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,1,0,0,0,-1,0,0,0,0,0,0,1,1,0,0,1,0],[0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0],[0,0,0,1,0,0,-1,0,1,0,0,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,-1,-1,0,0,0,0],[1,0,0,0,0,0,-1,1,0,0,0,0,-1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0],[0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,-1,0,0,-1,0,0,0,0,0,-1,1,-1,0,0,0,0,1,0,0,0,-1,0,1,0,0,0,-1,0,1,1],[0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,0,0,-1,0,-1,0,0,0,1,0,0,-1,0,0,-1,0,1,0,-1,0,1,0,1,0,-1,-1,1,0,1,0],[0,0,1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,1],[0,-1,0,0,0,0,0,0,0,1,0,-1,0,1,0,0,0,0,1,-1,0,0,1,0,0,0,0,0,0,0,0,0],[0,-1,0,-1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,1],[0,0,0,0,0,1,0,0,1,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,0,0,1,0,-1,0,0,0,-1,0,0,0,0,1,0,0,1,-1,0,0,1,0,-1,0,1],[0,0,0,1,0,0,1,0,-1,0,0,1,-1,0,-1,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0],[0,0,0,0,1,1,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,1,0,0,0,1,0,0,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0],[0,0,-1,-1,0,0,0,-1,-1,0,0,0,0,-1,0,0,0,-1,0,-1,0,-1,-1,0,0,0,0,0,0,0,0,0],[-1,0,-1,-1,0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,0,-1,-1,0,0,0,0,1,0,1,0,0],[0,0,-1,0,0,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1,1,0,-1,0,1],[0,0,0,0,0,0,-1,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,-1,1,-1,0,0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,0,0,0,1,0,0,1,0],[0,-1,0,-1,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0],[1,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,-1,0,0,1,-1,0,0,0],[0,-1,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1],[0,0,0,-1,0,0,0,-1,0,-1,0,1,1,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,1],[0,0,0,0,0,0,0,0,0,-1,0,0,0,-1,0,0,0,1,0,0,1,0,0,-1,0,0,0,0,-1,-1,0,0],[0,-1,-1,0,0,0,0,0,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,1,0,-1],[0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1],[0,-1,-1,0,1,-1,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0],[0,1,0,1,0,0,-1,0,0,-1,0,0,-1,0,0,0,0,-1,0,1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,-1,0,0,-1,0,0,0,0,0,0,0,-1,-1,0,0,0,1,0,0,0,0,-1,0,1],[0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,-1,1,1],[0,0,-1,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,-1,0,1,0,0,0,1,0,-1,-1,0,0,0],[-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,1,0,0,0,-1,0],[0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,1,0,0,0,0,1,1,-1,0,1,0,-1,0,0,0,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0],[-1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0],[-1,-1,0,-1,0,0,0,-1,-1,1,0,0,0,0,-1,0,0,1,0,0,-1,0,-1,0,0,1,0,0,-1,1,0,0],[0,-1,0,0,0,0,-1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,-1,0,0],[0,0,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,0,0,0,1,0,-1,0,1,0,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,-1,0,0,1,0,0,0,0,0,0,0]]
weights['h6']=[[0,0,-1,1,0,0,0,0,-1,0,1,0,0,0,0,0],[0,0,0,0,0,1,0,0,-1,0,0,-1,0,0,0,0],[0,0,-1,0,0,0,-1,0,0,0,0,0,0,-1,0,0],[0,0,0,0,1,0,-1,0,-1,0,1,0,0,0,0,-1],[1,0,-1,0,1,1,-1,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,1,-1,-1,0,0,0,0,0,-1,0,0],[0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0],[0,0,-1,0,0,0,-1,0,-1,0,0,0,0,0,0,0],[1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,1,-1,1,-1,1,-1,1],[0,0,0,1,0,1,0,0,0,-1,-1,0,-1,0,0,0],[-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,1,1,0,-1,0,0,0,0,0,0,0,1,0],[0,0,0,0,-1,0,1,1,1,0,0,0,1,0,0,0],[0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0],[0,1,0,0,0,0,-1,-1,-1,0,1,-1,1,0,0,0],[0,1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,1,1,0,-1,0,-1,0,0,0,0,0,1,0],[0,0,0,1,0,1,0,-1,0,0,1,-1,0,-1,0,0],[0,-1,0,0,0,1,-1,0,-1,0,1,0,0,0,0,0],[0,0,0,0,0,1,0,-1,0,0,0,0,-1,0,0,0],[1,0,-1,0,0,0,0,0,-1,-1,1,0,0,-1,0,0],[-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],[-1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0],[0,0,0,0,-1,0,1,0,0,1,0,0,0,0,0,0],[-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,0,0,1,0,-1,0,0],[1,-1,0,0,0,1,0,0,0,0,0,0,0,-1,0,1],[0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0],[0,0,0,-1,0,-1,0,1,0,1,0,0,0,0,0,1]]
weights['h7']=[[0,0,0,0,0,0,0,1],[0,0,0,0,0,1,-1,0],[1,0,0,0,1,1,-1,0],[0,0,0,0,-1,-1,0,1],[0,0,0,-1,0,0,0,0],[-1,0,0,-1,0,0,0,0],[0,0,0,0,1,1,0,0],[0,-1,0,1,0,0,0,0],[0,0,-1,1,-1,1,0,0],[0,-1,0,0,0,0,0,0],[-1,0,0,0,0,0,0,1],[0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,-1,0],[1,0,0,0,1,1,0,0],[-1,0,0,0,0,0,1,0],[0,-1,0,0,0,1,0,0]]
weights['h8']=[[0,0,0,0],[0,0,0,-1],[-1,0,0,0],[0,0,-1,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,-1,0,0]]
weights['ot']=[[1,-1],[1,-1],[-1,1],[1,-1]]
biases['h1']=[-0.75838631,0.22325644,0.3661513,0.04274797,-0.18642892,-1.91277719,-0.80803406,-0.01960801,0.24396485,0.85174441,0.08960561,1.6038605,-0.85399091,0.53795105,0.45613816,0.43826708,-1.24274254,0.25113568,0.43070152,2.08145308,0.85167861,1.74273098,0.3259702,-0.49902669,-1.57034206,0.4466061,-0.16187885,-0.86081558,0.38545319,0.08060583,0.46095935,0.23683305,0.34784314,-0.75685036,0.67949331,1.09625065,-0.15084247,0.16492933,0.0051393,-0.56413305,0.91292554,0.56131333,-0.52946186,0.12506227,0.49027461,-0.75240505,0.84587771,-1.5862304,0.46839416,-0.63216043,0.02510483,1.71853042,1.15251303,0.18544617,0.56987059,0.56021881,-0.31478375,0.59978819,-0.0390451,1.26714098,-0.20711727,-1.29281199,0.32811761,0.86572182,-0.55016732,0.52765387,-0.39353991,0.10292181,-1.60755622,-1.23448658,0.87691373,0.33037448,-0.6851207,0.01629753,-0.56425387,-0.08128459,0.85735267,-0.18193775,0.37988979,-0.64138383,0.61591107,1.99870193,0.28177643,-1.33806908,0.37232378,-0.06622417,-0.16629213,-0.79892159,0.34197026,0.24093334,0.15845989,1.99022806,0.41713652,0.36147329,-0.54348457,0.34965312,0.68893027,0.40655792,-0.50279969,0.99531108,1.14496732,0.50115263,1.0855397,0.21203953,0.36224124,-0.44545016,-0.06108502,-0.42252547,0.31079993,0.15124306,-0.171984,0.22376262,-0.27698132,-0.51138431,0.36592683,0.04137823,-0.57388896,0.76595682,-0.22181383,0.42178982,-0.16694742,-0.00779887,1.0073663,-0.73121285,0.51336205,-0.04471327,1.41548264,-1.08178341,-0.62341046,-0.12372742,0.27112284,-0.5425542,0.02683583,-0.09060729,0.25593004,-0.36430475,0.47513971,-1.13346016,-0.09734876,-0.9836359,0.4812057,-1.09396338,-1.68222177,0.26747483,-1.07827926,1.48726964,0.04227775,-0.41416174,0.42192128,0.26364598,-0.06502299,-0.65150225,-0.21705605,0.01762162,-0.03315261,-0.03575878,0.87493867,1.13863266,-0.67741919,-0.14907642,0.48039198,-0.1308658,0.99266887,0.58915162,-0.27365562,0.41964144,1.37773824,0.20602435,0.14860664,1.18284798,-0.70387924,1.49424016,0.72642297,-0.20956656,0.4014425,-0.01680757,0.35934851,0.65991396,0.2276686,-0.80471152,1.42717957,-0.03788657,0.66918284,0.3373583,-0.55328232,0.3595891,0.01037093,-0.74202389,0.53921366,-0.14283738,0.28817052,0.47769594,-0.39782754,0.61838245,-0.23351997,-1.11698985,0.22446744,0.04633033,-0.12669152,-0.18049201,0.93857402,0.72753316,1.01914775,-0.54275161,-0.22227208,-0.32806891,-0.13832167,0.63521934,-0.51217824,0.07397119,-0.37507373,0.93402857,0.30924571,-0.65221202,-0.16693161,1.62810624,-0.62020791,0.8301369,1.00028539,-0.6777941,-0.96170092,0.58973277,-0.31483132,-0.51082367,-0.97357213,-0.49850401,-0.5289076,1.81171882,1.02543473,-0.31089318,-0.12872607,0.16354108,0.36913899,1.09225893,0.57854813,-0.47853771,-0.5945968,0.15901639,-0.10812723,-0.45674896,-0.55657256,-0.14822146,-0.11402988,0.53377521,0.0620095,0.60633856,0.51557344,0.05082776,-0.19991833,-0.72331309,-0.09360424,-0.10621011,0.88094985,-0.66981059,0.04000818,0.42387447,-0.7095558,0.229497,0.1336575,-0.39735314,-0.83383441,0.06678333,0.10513174,-0.4581717,0.39024681,0.83301842,-0.49177831,-0.10640224,-0.859936,0.71730208,-0.21621805,-0.81033719,0.30590087,-0.81643927,0.39619124,-0.54814494,0.0389558,-0.18326265,1.0034312,0.31479606,-0.44148341,-1.25459862,-1.10964215,-0.41614726,0.04627812,-1.06168759,0.02002322,-0.72979659,-1.93277991,-0.51986146,0.52620125,0.84253758,1.53212392,0.09363434,-0.03403207,0.30562013,0.55891013,0.89932775,0.02245009,0.63404137,-1.80523932,-0.40638718,0.68871969,0.82206827,-0.38637623,-0.73473322,-0.37002009,-0.30271611,-0.47091982,0.82862508,-0.43188041,-0.54493034,0.41517314,0.145335,-0.67550671,0.70510811,-0.69677651,-0.70649147,0.07563536,1.31660688,0.59376049,-0.58528829,0.58631963,0.37290329,0.35244054,0.30067334,0.96231318,-0.38308349,0.3364771,0.2572968,0.23443055,-0.33822718,1.1036644,-0.20730796,0.616243,0.44198009,-0.0201081,-0.16173477,0.4277764,-0.69046903,0.47022668,-0.31821236,1.04534817,-0.16596061,-0.16297126,-0.59489846,0.08836143,0.1393066,-0.86073482,-0.73148203,-0.66801763,-0.62742537,0.04225682,0.01075286,0.96943927,-0.66026026,0.85500878,-1.04753351,-0.64488912,0.02439118,-1.82720101,0.22494256,-0.23329106,0.60326082,0.77737594,-0.3786011,-0.66095525,-0.5337832,-1.28802109,-0.56343079,-1.9196105,0.73249811,0.61279941,-0.00397693,-0.52629685,-0.75938362,0.20460056,-1.8570199,-0.68861771,0.15859851,0.21411499,0.60947013,-0.41565868,0.39626074,0.15126713,1.28947139,0.48289996,0.07489935,0.0283778,0.02435057,0.45891711,0.11613286,-0.13962345,-0.36340398,0.98764366,0.14538153,0.50448447,0.56551468,0.89349157,0.65429753,-0.93199235,1.35361648,-0.83061594,0.01724045,0.11651736,-0.27514946,0.99062365,0.02618893,0.34353986,1.05287516,-1.20031178,-0.92302966,-0.04824307,-0.13601053,1.58544779,-0.13461575,0.32398349,1.07883072,0.56748331,-0.05382666,0.36194202,0.75287676,0.5748468,0.37881598,-0.28068805,-0.15569983,-0.68184352,-0.46124873,-0.55365366,-0.22657661,-0.30390725,-0.18583784,0.64373571,-0.81524283,1.2125628,-0.3518934,0.02502261,-0.01220285,-0.87484765,-0.18078138,-0.82611585,-0.61129063,0.45169732,-1.16116047,0.46084613,0.87120134,0.2265529,0.50698334,0.7848832,-0.91899204,1.11844838,-0.67931521,-1.00208187,0.88176942,-0.48386562,0.28318009,0.02742935,-0.51224506,1.51856816,0.99641907,0.2211637,0.47317171,-0.09139436,-0.09256369,-0.16263789,-0.47429195,-0.1838042,0.41161099,-0.9517855,1.83000433,0.73172688,-0.10265669,-0.86623639,-0.23994513,0.07114528,0.16666174,0.37680474,1.2879554,-0.56163168,0.28567949,-0.02690178,-1.17495346,-1.10606802,0.38842198,0.15740207,-0.41672951,-0.14299503,1.37578356,-0.2749992,0.55720222,-0.17567132,0.84262663,0.32651588,-0.74215895,-0.05245689,-1.27480698,0.73289984,-0.18626492,0.84428477,0.71651721,0.15786526,0.21483867,0.10011235,-1.48081684,0.03249017,-1.09803367,-0.73202813,-0.24716856,-1.0122596,0.44106114,-0.26405057,0.20990339]
biases['h2']=[-1.05342436e+00,-1.39980316e+00,2.03159213e-01,1.05148196e+00,-1.80672705e+00,1.82436585e-01,-1.13444865e+00,-1.82042193e+00,2.06581521e+00,-3.58223772e+00,6.95828378e-01,-5.08111775e-01,-2.74868757e-01,3.42273092e+00,8.63203127e-03,-1.83289386e-02,1.45377207e+00,-1.15617990e+00,5.64584196e-01,1.59599364e+00,1.69427586e+00,2.62676686e-01,-1.88102022e-01,1.32927203e+00,-1.95437372e+00,2.42104602e+00,7.40313232e-01,6.06723607e-01,-1.38821885e-01,-1.60850644e+00,-1.72351241e+00,7.98765838e-01,-4.91557419e-01,3.04781818e+00,3.60216707e-01,-6.20082796e-01,2.89737606e+00,1.17092967e+00,3.90403897e-01,3.18431944e-01,5.54894447e-01,1.87176108e+00,3.33619088e-01,1.07266855e+00,-7.56016135e-01,1.98620558e+00,-1.20083404e+00,1.18970668e+00,-5.98169422e+00,-9.33716893e-01,1.94450021e+00,1.37994623e+00,3.34229279e+00,1.24305561e-01,-2.11481357e+00,-1.43526077e+00,6.12051785e-01,-8.70336115e-01,-8.95630658e-01,-8.66151333e-01,2.32753348e+00,1.29964852e+00,-1.49839115e+00,-5.55120945e-01,2.27861071e+00,9.50749636e-01,7.77384937e-01,-8.73512149e-01,-1.41659534e+00,1.75535595e+00,1.29518285e-01,1.13430239e-01,6.27170280e-02,6.72140956e-01,-2.29641151e+00,6.49462342e-01,2.29301238e+00,3.89905572e+00,1.48809537e-01,1.81189823e+00,5.95065057e-01,-2.15831375e+00,-1.21339393e+00,9.26565588e-01,2.50170350e+00,9.83361900e-01,-2.74019003e+00,-1.27375138e+00,9.45819259e-01,-6.71348572e-01,3.86696368e-01,9.36349511e-01,8.35462034e-01,-2.62477446e+00,1.29373205e+00,-1.05550897e+00,3.99545240e+00,-4.54641283e-01,-2.76912004e-02,-1.74462616e+00,1.70206165e+00,-2.41892004e+00,-1.55639374e+00,-1.51815677e+00,-3.46113300e+00,3.31397915e+00,-1.24644093e-01,1.62035644e+00,1.51594687e+00,1.68226492e+00,-1.88201189e+00,-5.71816921e-01,1.75745666e+00,1.85710895e+00,-1.68436992e+00,-3.08905196e+00,5.43756676e+00,1.45396864e+00,-1.05315232e+00,1.74994099e+00,-9.40820098e-01,1.51054358e+00,-1.20512319e+00,-9.55956429e-02,2.52830148e+00,2.34718800e+00,-1.99379027e+00,-3.52838933e-01,3.00426245e-01,-8.26654017e-01,-1.93501425e+00,-1.58191073e+00,-7.02108383e-01,-1.08783007e+00,2.66556859e+00,-1.38825309e+00,5.16254187e+00,3.84304619e+00,2.15173244e+00,1.31889194e-01,1.24909019e+00,4.87285584e-01,-1.26585686e+00,1.57890999e+00,-1.74500588e-02,-9.35851932e-01,-7.81627655e-01,-3.53721476e+00,-2.60330868e+00,-7.97933862e-02,2.83598804e+00,-5.59481919e-01,-4.52595800e-01,-1.83058038e-01,1.12851405e+00,-1.59746432e+00,1.40051103e+00,-1.24016369e-03,2.61948705e-01,1.32358479e+00,5.12392595e-02,1.12351455e-01,-4.99467045e-01,1.68335032e+00,-1.97782230e+00,-1.62648141e+00,-1.95827794e+00,-1.11467779e+00,-6.10569537e-01,-3.44152153e-01,-2.43009257e+00,1.50815165e+00,-1.18140781e+00,-1.21971834e+00,-2.90980220e+00,2.01824617e+00,-1.16622269e+00,1.00939810e+00,3.13437939e+00,1.16176856e+00,5.67750335e-01,1.14894831e+00,3.85866374e-01,-1.94976044e+00,-4.60594743e-01,-2.67368484e+00,3.92775118e-01,-8.67723763e-01,4.97550845e-01,1.66968429e+00,2.82347989e+00,-5.97119331e-01,1.49358797e+00,7.56496727e-01,-3.04872537e+00,4.16295588e-01,-8.05917621e-01,-1.73112988e+00,1.58506072e+00,1.46411002e+00,1.39309630e-01,-1.57582545e+00,-2.17992687e+00,-9.03384745e-01,1.10034847e+00,2.59618568e+00,1.72992516e+00,2.95751286e+00,1.17703474e+00,3.12278581e+00,2.06798720e+00,9.18568313e-01,7.53350198e-01,-1.72925723e+00,3.72874594e+00,-1.59395874e+00,-5.10369122e-01,-1.08896069e-01,1.10410702e+00,-2.26071692e+00,2.20656610e+00,-7.07889915e-01,2.87445641e+00,6.31533861e-01,1.43533301e+00,6.94648743e-01,-3.54844891e-03,-2.15368128e+00,6.76369607e-01,4.51184124e-01,-2.64637887e-01,1.22601712e+00,1.01479220e+00,1.19074070e+00,1.88120651e+00,1.03840637e+00,2.89848059e-01,1.79426622e+00,6.48150802e-01,-1.20275342e+00,-9.55022633e-01,-2.00721681e-01,-1.64444959e+00,-4.36392456e-01,-1.68646073e+00,6.91860199e-01,-9.50595856e-01,-1.04888833e+00,-1.32370031e+00,-1.48087788e+00,-1.63034964e+00,-3.86204451e-01,2.40476203e+00,-9.93393123e-01,4.44264531e-01,-8.10198933e-02]
biases['h3']=[-1.13363624,0.32613033,0.61433536,-0.22239061,0.48412296,1.25649369,-1.02547753,-0.04618088,0.14086641,-0.71543348,-2.94335222,2.13379192,1.94498205,-2.1709106,1.24989974,0.77102941,2.36732411,1.32222486,0.5146094,-0.11553077,-1.03146422,0.62719536,1.31774211,-0.96124679,-2.50988555,-0.45366779,0.38032675,-2.91422129,0.58530039,0.87143511,-0.96120816,0.27424228,0.48994577,0.26250941,-0.39985073,1.66635942,1.58435416,1.97424507,1.06304657,-1.27474725,-0.4898451,1.37862897,2.46789765,-1.90408337,0.31830043,-0.24786705,-1.86542678,0.31247732,0.37703493,0.26142538,-2.73049355,-0.26571617,-0.79116678,-0.77097493,0.89284831,-2.14048171,1.13537705,-0.0386041,0.45073104,-0.44674334,-1.0864166,0.27349648,1.07875562,0.92907321,1.03520799,0.18513943,0.84602189,0.17230186,-0.8165009,-0.12075548,0.8676492,-1.56394601,1.1820339,-0.30853039,0.12051687,0.68176013,-0.82725346,-0.03319294,-0.12738207,0.39093378,0.29597074,0.7226398,-1.42419004,0.38557556,0.84964567,-0.62778902,-0.0778188,-1.73500037,0.28430077,0.30419424,0.38397872,-1.17517197,0.31424892,-0.35970017,0.2829107,-0.19991423,-0.78475726,-1.65990233,0.21067052,0.92086095,0.87730813,-1.73240232,-2.35715294,0.56993318,1.75070858,-0.17105484,0.7194373,-0.20133941,1.09704649,-1.69043469,0.22463049,-0.55070692,-0.26134074,0.19300373,1.20172369,-1.08701289,0.39097968,-0.06686087,1.16240728,0.4712404,0.54454613,1.1076175,0.52594656,-0.05399223,-0.17212254,-0.49627194,-0.79934871,1.63909209]
biases['h4']=[-1.48487401,0.02767679,-1.30065572,0.65488893,-0.15545654,0.08067607,0.9737305,-1.62487626,1.43290079,0.30680951,0.25399256,-0.67659688,-0.46591333,-0.83569205,1.02750206,0.80867636,1.4127599,-1.06923652,-0.1223136,-2.31868434,-3.09726286,2.32044244,-1.65504718,-1.62685108,-1.10638714,-0.52965975,-1.58411431,0.8376776,-0.43856585,-0.53859115,-0.59795642,0.06152153,1.23426402,-1.58160186,-1.35682845,-1.47763491,-2.50431633,-1.99291241,-0.97384143,0.55271834,0.2870726,0.22317782,0.78912771,-1.15618622,0.4874211,-1.25146949,-0.10777503,-0.55757761,0.88772124,0.46356371,-0.32389593,0.76377964,-0.9119733,0.45707193,0.9154579,0.41803268,0.44890141,1.32998407,-0.98155272,-1.71039069,2.0507524,-1.09597385,0.52860981,0.87618786]
biases['h5']=[0.05284709,-0.16946667,-1.43906462,-1.96511889,-0.57431805,0.84686089,0.90824252,-3.36136508,-1.48145485,1.08798373,-0.01615258,1.89562583,0.1237179,-2.2727263,0.23072381,1.64347029,-0.42192987,1.38093436,-0.39141071,-0.98737907,0.46094498,0.54973739,0.24465215,-0.62899661,0.14871517,1.03046119,-0.63425088,1.41174507,0.11092227,-0.57779527,-1.65051389,0.43125758]
biases['h6']=[0.34451357,-0.77362674,0.38224435,1.38175106,-5.27981234,1.98447251,-0.9805094,0.79063934,1.05371106,0.62452036,-0.07825523,-0.7189731,0.00693204,0.28667098,-0.95992249,-0.44849247]
biases['h7']=[2.08720493,-1.25405192,-0.62508184,-1.30268133,1.08989024,-1.3406539,0.80394268,0.27874357]
biases['h8']=[1.14074838,1.30791283,1.53070736,-0.79740298]
biases['ot']=[-0.26179692,0.26179692]

layer_keys = list(n)
layer_keys.remove('in')
layer_keys.remove('ot')
layer_keys = sorted(layer_keys, key=lambda x : int(x[1:]))    # key is the single- or multi-digit number after the first character
layer_keys.append('ot')    # layer_keys now starts with the first hidden layer upto the last and then the output layer

def multilayer_perceptron(inputs, weights, biases):
    layer['in'] = inputs
    prev_key    = 'in'
    for key in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'ot']:
        layer[key] = tf.tanh(tf.add(tf.matmul(layer[prev_key], tf.constant(weights[key], dtype=tf.float32)),
                                    tf.constant(biases[key])))
        prev_key   = key
    return layer['ot']

pred         = multilayer_perceptron(x, weights, biases)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy     = tf.reduce_mean(tf.to_float(correct_pred))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Classification metrics:', classification_metrics(features_test, labels_test))